___
___
## <font color='coral'>**Appel de l'environment** </font> 

In [1]:
import ipywidgets as widgets
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt

from scipy import special
import scipy.integrate as integrate
from scipy.misc import derivative
import time
from scipy.interpolate import InterpolatedUnivariateSpline
import glob
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
from scipy.ndimage.filters import uniform_filter1d
from scipy.ndimage.filters import gaussian_filter1d
from scipy.signal import savgol_filter
from multiprocessing import Pool

## For sliders
from ipywidgets import Layout, interact, IntSlider
from ipywidgets import interact, interactive, interactive_output, HBox, VBox

## For color cyclers
import itertools
from cycler import cycler

mpl.rcParams['image.cmap'] = 'RdYlBu_r'

plt.close('all')
plt.rcParams['axes.formatter.limits'] = (-3,1)
plt.rc('font',size=18)
plt.rcParams['lines.linewidth'] = 2
plt.rcParams.update({"axes.grid" : True})

#Size of figures
laptop    = 6
cadarache = 12
home      = 16

xfigsize  = laptop

dpi_laptop = 50
dpi_cadarache = 65
dpi_home = 100
dpi_custom = dpi_home

def Fourier2D(F0, y0, x0):
    nx0 = len(x0)
    nx  = 2 * int(nx0 / 2)
    hnx = int(nx / 2)
    ny0 = len(y0)
    ny  = 2 * int(ny0 / 2)
    hny = int(ny / 2)

    x = x0[0:nx]
    y = y0[0:ny]
    F = F0[0:ny, 0:nx]

    Lx   = x[nx - 1] - x[0]
    dx   = x[1] - x[0]
    dkx  = 2. * np.pi / (Lx + dx)
    kx   = np.zeros(nx)
    temp = -dkx * np.r_[1:hnx + 1]
    kx[0:hnx]  = temp[::-1]
    kx[hnx:nx] = dkx * np.r_[0:hnx]

    Ly   = y[ny - 1] - y[0]
    dy   = y[1] - y[0]
    dky  = 2. * np.pi / (Ly + dy)
    ky   = np.zeros(ny)
    temp = -dky * np.r_[1:hny + 1]
    ky[0:hny]  = temp[::-1]
    ky[hny:ny] = dky * np.r_[0:hny]

    TFF = np.zeros((ny, nx), dtype=complex)
    AA  = np.zeros((ny, nx), dtype=complex)
    var = np.conjugate(np.fft.fft2(np.conjugate(F))) / float((nx * ny))

    AA[:, 0:hnx]   = var[:, hnx:nx]
    AA[:, hnx:nx]  = var[:, 0:hnx]
    TFF[0:hny, :]  = AA[hny:ny, :]
    TFF[hny:ny, :] = AA[0:hny, :]

    return TFF, kx, ky

def Fourier1D(F0, x0, axis=-1):
    """ Personal FFT1D function"""
    x0 = np.asarray(x0)
    assert x0.ndim == 1

    nx0 = len(x0)
    nx  = 2 * int(nx0 / 2)
    hnx = int(nx / 2)

    dx = x0[1] - x0[0]
    kx = 2 * np.pi / dx * np.linspace(-.5, .5, nx, endpoint=False)

    TFF  = np.conjugate(np.fft.fft(
        np.conjugate(F0),
        n=nx, # crop `axis` at length `nx`
        axis=axis
    ))
    TFF /= nx
    TFF  = np.fft.fftshift(TFF, axes=axis)

    return TFF, kx

## <font color='coral'>**Chargement des données** </font> 

In [2]:
import h5py

def hdf5_to_dict(filename):
    """Load a dictionary of arrays and strings as unicode characters from an HDF5 file."""
    with h5py.File(filename, 'r') as f:
        d = {}
        for k in f.keys():
            v = f[k][()]
            if isinstance(v, bytes):
                d[k] = v.decode('utf-8')
            else:
                d[k] = v
    return d

qprof_05 = hdf5_to_dict('data/TS45511_q05.hdf5'); qprof_05['title']=r'$q_{0.5}$'; qprof_05['c']='xkcd:electric blue'; qprof_05['ls']='-'
qprof_1  = hdf5_to_dict('data/TS45511_q1.hdf5'); qprof_1['title']=r'$q_{1.0}$'; qprof_1['c']='xkcd:bluish purple'; qprof_1['ls']='-'
qprof_15 = hdf5_to_dict('data/TS45511_q15.hdf5'); qprof_15['title']=r'$q_{1.5}$'; qprof_15['c']='xkcd:watermelon'; qprof_15['ls']='-'

qprof_05_n0 = hdf5_to_dict('data/TS45511_q05_n0.hdf5'); qprof_05_n0['title']=r'$q_{0.5}$ - $n=0$'; qprof_05_n0['c']='xkcd:cobalt blue'; qprof_05_n0['ls']='-'
qprof_15_n0 = hdf5_to_dict('data/TS45511_q15_n0.hdf5'); qprof_15_n0['title']=r'$q_{1.5}$ - $n=0$'; qprof_15_n0['c']='xkcd:dark red'; qprof_15_n0['ls']='-'

#qprof_gdp = pickle.load(open('data/gdp_ti0_te597900.pkl','rb')); qprof_gdp['title']=r'gdp'; qprof_gdp['c']='darkorange'

qprof_05new = hdf5_to_dict('data/TS45511_q05new.hdf5'); qprof_05new['title']=r'$q_{0.5}^{NEW}$'; qprof_05new['c']='xkcd:lightish blue'; qprof_05new['ls']='-'

qprof_05_br = hdf5_to_dict('data/TS45511_q05_br.hdf5'); qprof_05_br['title']=r'$q_{0.5}^{BR}$'; qprof_05_br['c']='xkcd:blue'; qprof_05_br['ls']='-'
qprof_075_br = hdf5_to_dict('data/TS45511_q075_br.hdf5'); qprof_075_br['title']=r'$q_{0.75}^{BR}$'; qprof_075_br['c']='xkcd:pink'; qprof_075_br['ls']='-'
qprof_15_br = hdf5_to_dict('data/TS45511_q15_br.hdf5'); qprof_15_br['title']=r'$q_{1.5}^{BR}$'; qprof_15_br['c']='xkcd:red'; qprof_15_br['ls']='-'

all_sim = [qprof_05, qprof_1, qprof_15, qprof_05_n0, qprof_15_n0, qprof_05new, qprof_05_br, qprof_075_br, qprof_15_br]

print(qprof_05_br.keys())

daltonian = 0
if daltonian:
    import seaborn as sns
    qprof_05['c']= sns.color_palette("colorblind")[0]
    qprof_1['c'] = sns.color_palette("colorblind")[2]
    qprof_15['c']= sns.color_palette("colorblind")[3]
    
    qprof_05_n0['c']= sns.color_palette("colorblind")[0]
    qprof_15_n0['c']= sns.color_palette("colorblind")[3]

iaea = 0
if iaea:
    qprof_05['c']= 'xkcd:blue'
    qprof_1['c'] = 'xkcd:green'
    qprof_15['c']= 'xkcd:black'
    
soutenance = 1
if soutenance:
    qprof_05['title'] = r'$q_{ref} \times 0.5$'
    qprof_1['title']  = r'$q_{ref}$'
    qprof_15['title'] = r'$q_{ref} \times 1.5$'
    
## Colors
qprof_05['c2'] = 'xkcd:cornflower blue'
qprof_05['c3'] = 'xkcd:apple green'
qprof_05['c4'] = 'xkcd:melon'
qprof_1['c2'] = 'xkcd:grassy green'
qprof_1['c3'] = 'xkcd:carmine'
qprof_1['c4'] = 'xkcd:strong blue'
qprof_15['c2'] = 'xkcd:reddish'
qprof_15['c3'] = 'xkcd:cerulean'
qprof_15['c4'] = 'xkcd:darkish green'
    
#q_ar = [qprof_05,qprof_1,qprof_15,qprof_05_n0,qprof_15_n0,qprof_05new]
## Check length of arrays and correct if necessary
for iq,q in enumerate(all_sim):
    #print('Time array length = %s' % len(q['time']) )
    for p in q.keys():
        if type(q[p]) is np.ndarray:
            if q[p].ndim == 2:
                #if len(q[p][:,0]) != len(q['time']):
                    #print('Time dimension length of %s = %s' % (p,len(q[p][:,0])) )
                if len(q[p][:,0]) == len(q['time']):
                    #print('Time dimension length of %s = %s' % (p,len(q[p][:,0])) )
                    q[p] = q[p][:-1,:]
    q['time'] = q['time'][:-1]
    
## Defining custom values
for iq,q in enumerate(all_sim):
    q['gradT']             = np.gradient(q['T'],q['rg'],axis=1)
    q['Ptransfert']        = -q['dRSpoldr'] * (-q['Er'])
    q['Ptransfert_w_prefactor']  = -q['dRSpoldr'] * (-q['Er']) / (1 + 2 * q['q']**2)
    q['Ptransfert_vE']     = -np.gradient(q['RSpol_vE'],q['rg'],axis=1) * (-q['Er'])
    q['Ptransfert_vD']     = -np.gradient(q['RSpol_vD'],q['rg'],axis=1) * (-q['Er'])
    q['Ptransfert_approx'] = q['RSpol'] * (-q['dErdr'])
    q['dErdt']             = np.gradient(q['Er'],q['time'],axis=0)
    q['dIsqdr']            = np.gradient(q['turb_int_sq_avg'],q['rg'],axis=1)
    q['dIdr']              = np.gradient(np.sqrt(q['turb_int_sq_avg']),q['rg'],axis=1)
    q['Jr']                = -(q['Gamma_vE']+q['Gamma_vD'])
    q['Jr_vD']             = -q['Gamma_vD']
    q['Jr_vE']             = -q['Gamma_vE']
    q['neo_energy_evol']   = -q['Ptransfert'] + 0.5*np.gradient(-q['Er']**2,q['time'],axis=0)
    q['neo_energy_evol_w_prefactor'] = -q['Ptransfert_w_prefactor'] + 0.5*np.gradient(-q['Er']**2,q['time'],axis=0)
    q['tevol_VEsq']        = 0.5*np.gradient(-q['Er']**2,q['time'],axis=0)

dict_keys(['Er', 'Gamma_vD', 'Gamma_vE', 'Gamma_vEn0', 'Gamma_vEndiff0', 'IturbGC_FSavg', 'Phi00', 'Phi01', 'Ppar', 'Pperp', 'Qpar_vD', 'Qpar_vE', 'Qpar_vEn0', 'Qperp_vD', 'Qperp_vE', 'Qperp_vEn0', 'Qtot', 'Qtot_vD', 'Qtot_vE', 'Qtot_vEn0', 'RSphi', 'RSphi_vD', 'RSphi_vE', 'RSphi_vEn0', 'RSpol', 'RSpol_vD', 'RSpol_vE', 'RSpol_vEn0', 'T', 'VP', 'VT', 'dErdr', 'dPhi01dr', 'dRSphidr', 'dRSpoldr', 'dVPdr', 'dVTdr', 'delta_n_avg', 'eps', 'n', 'nIturbGC_FSavg', 'nu_i', 'nustar', 'q', 'rg', 'rho', 'spreading_0deg', 'spreading_180deg', 'spreading_270deg', 'spreading_90deg', 'spreading_FSavg', 'time', 'turb_int_avg', 'turb_int_sq_avg', 'title', 'c', 'ls'])


___
___

## <font color='xkcd:sage'>**Exploration**</font> - Profils radiaux de quantités choisies

In [47]:
plt.close('all')

q_ar = [qprof_05_n0,qprof_15_n0]
param_ar = ['Phi00','Phi01','dPhi01dr','Er','VP','dErdr','RSpol','RSphi','dRSphidr','VT','nustar']
param_ar = ['Gamma_vD', 'Gamma_vE', 'Gamma_vEn0', 'Gamma_vEndiff0']

param_ar = ['Er']

for ip,p in enumerate(param_ar):
    fig = plt.figure(figsize=(5,5))
    fig.suptitle(p)
    ax1 = fig.add_subplot(111)
    for iq,q in enumerate(q_ar):
        
        irhomin = (np.abs(q['rho'] - 0.1 )).argmin()
        irhomax = (np.abs(q['rho'] - 1.0 )).argmin()

        itmin = (np.abs(q['time'] - 40.0e3 )).argmin()
        itmax = (np.abs(q['time'] - 60.0e3 )).argmin()
        
        ax1.plot(q['rho'][irhomin:irhomax], np.mean( q[p][itmin:itmax,irhomin:irhomax],axis=0),ls=q['ls'],c=q['c'],label=q['title'])
        
        ax1.axhline(y=0,ls='--',c='k')
        
    ax1.legend()
    ax1.set_xlabel(r'$r/a$')
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## <font color='orangered'>**(En cours)**</font> <font color='xkcd:sage'>**Exploration**</font> - Profils radiaux de quantités choisies v2

In [9]:
plt.close('all')

import matplotlib
matplotlib.rcParams['figure.figsize'] = [10, 10] # for square canvas
matplotlib.rcParams['figure.subplot.left'] = 0
matplotlib.rcParams['figure.subplot.bottom'] = 0.5
matplotlib.rcParams['figure.subplot.right'] = 1
matplotlib.rcParams['figure.subplot.top'] = 1

fig = plt.figure(figsize=(12,6))
ax_rprof = fig.add_subplot(111); ax_rprof2 = ax_rprof.twinx()

def update_plot(N,dpi_adjust,param,param2,float_range_time,
               q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool,
               q05br_bool,q075br_bool,q15br_bool):
    
    q_ar = [qprof for qprof, qprof_bool in zip(all_sim, [q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool,q05br_bool,q075br_bool,q15br_bool]) if qprof_bool]
    
    tmin_avg,tmax_avg = float_range_time

    ## Set dpi of figure
    if dpi_adjust!=dpi_custom:fig.set_dpi(dpi_adjust)

    ## Clear previous plot
    ax_rprof.clear(); ax_rprof2.clear()
    ax_rprof2.grid(False)
    
    fig.suptitle(r'Study of %s' %param)
    
    for q in q_ar:

        irhomin = (np.abs(q['rho'] - 0.01 )).argmin()
        irhomax = (np.abs(q['rho'] - 1.1 )).argmin()
        
        itmin_avg = (np.abs(q['time'] - tmin_avg )).argmin()
        itmax_avg = (np.abs(q['time'] - tmax_avg )).argmin()

        p_ma = uniform_filter1d(uniform_filter1d(q[param],size=N,axis=1),size=N,axis=1)
        
        ax_rprof.set_title(r'Radial profile (avg %.0f < t$[\omega_{c0}^{-1}]$ < %.0f)' % (tmin_avg , tmax_avg) )
        ax_rprof.plot(q['rho'][irhomin:irhomax], np.mean( p_ma[itmin_avg:itmax_avg,irhomin:irhomax],axis=0),ls=q['ls'],c=q['c'],label=q['title'])
        ax_rprof.set_xlabel(r'$r/a$')
        ax_rprof.axhline(y=0,c='k',ls='--')
        ax_rprof.axvline(x=1,c='k',ls='--')
        ax_rprof.legend()
        
        if param2:
            color1 = 'xkcd:cerulean'
            color2 = 'xkcd:reddish orange'
            p_ma2 = uniform_filter1d(uniform_filter1d(q[param2],size=N,axis=1),size=N,axis=1)
            
            ax_rprof2.plot(q['rho'][irhomin:irhomax], np.mean( p_ma2[itmin_avg:itmax_avg,irhomin:irhomax],axis=0),ls=q['ls'],c=q['c2'],label=q['title'])
            ax_rprof2.yaxis.label.set_color(color2)
            ax_rprof2.tick_params(axis='y', colors=color2)
        
        ax_rprof.plot(q['rho'][10:10+N],np.ones(N)*ax_rprof.get_ylim()[1]/4,c='xkcd:battleship grey',lw=5) #Visualize the sliding average window

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    
float_range_time_slider = widgets.FloatRangeSlider(value=[100e3, 145e3], min=0, max=150e3, step=1000, description='Time range', layout=Layout(width='50%'), style = {'description_width': 'initial'})
    
interact(update_plot, 
         N=widgets.IntSlider(min=1, max=100, step=1,value=1),
         float_range_time=float_range_time_slider,
         dpi_adjust=widgets.IntSlider(min=30, max=200, step=1,value=dpi_custom),
         param = widgets.Text(value='Er',placeholder='Type something',description='Select parameter:',disabled=False),
         param2 = widgets.Text(value='',placeholder='Type something',description='Select parameter:',disabled=False),
         q05_bool    = widgets.Checkbox(value=True,description='q05'),
         q1_bool     = widgets.Checkbox(value=False,description='q1'),
         q15_bool    = widgets.Checkbox(value=True,description='q15'),
         q05n0_bool  = widgets.Checkbox(value=False,description='q05n0'),
         q15n0_bool  = widgets.Checkbox(value=False,description='q15n0'),
         q05new_bool = widgets.Checkbox(value=False,description='q05new'),
         q05br_bool = widgets.Checkbox(value=False,description='q05br'),
         q075br_bool = widgets.Checkbox(value=False,description='q075br'),
         q15br_bool = widgets.Checkbox(value=False,description='q15br')
        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1, description='N', min=1), IntSlider(value=100, description='dpi_adjust…

<function __main__.update_plot(N, dpi_adjust, param, param2, float_range_time, q05_bool, q1_bool, q15_bool, q05n0_bool, q15n0_bool, q05new_bool, q05br_bool, q075br_bool, q15br_bool)>

## <font color='xkcd:sage'>**Exploration**</font> - Video of radial profiles

In [3]:
plt.close('all')
fig = plt.figure(figsize=(12,6))
ax_rprof = fig.add_subplot(111) 

def update_plot(N,dpi_adjust,param,
               q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool,
               q05br_bool,q075br_bool,q15br_bool,
               it_play, bool_fixed_y_range):

    ## Select profiles to plot
    q_ar = [qprof for qprof, qprof_bool in zip(all_sim, [q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool,q05br_bool,q075br_bool,q15br_bool]) if qprof_bool]

    ## Set title of figure
    fig.suptitle(r'Study of %s' %param)

    ## Set dpi of figure
    if dpi_adjust!=dpi_custom: fig.set_dpi(dpi_adjust)

    ## Clear previous plot
    ax_rprof.clear()

    for q in q_ar:

        ## Find index of rho=0.1 and rho=1.1
        irhomin = (np.abs(q['rho'] - 0.1 )).argmin()
        irhomax = (np.abs(q['rho'] - 1.1 )).argmin()

        ## Compute sliding average
        p_ma = uniform_filter1d(uniform_filter1d(q[param],size=N,axis=1),size=N,axis=1)

        ## Plot radial profile
        ax_rprof.set_title(r'Radial profile (at t= %.0f $[\omega_{c0}^{-1}]$)' % (q['time'][it_play]) )
        ax_rprof.plot(q['rho'][irhomin:irhomax], p_ma[it_play,irhomin:irhomax],ls=q['ls'],c=q['c'],label=q['title'])
        ax_rprof.plot(q['rho'][10:10+N],np.ones(N)*ax_rprof.get_ylim()[1]/4,c='xkcd:battleship grey',lw=5) #Visualize the sliding average window

        ymax = ax_rprof.get_ylim()[1]

    ## Plot reference lines
    ax_rprof.axhline(y=0,c='k',ls='--'); ax_rprof.axvline(x=1,c='k',ls='--')

    ## Set labels and legend
    ax_rprof.set_xlabel(r'$r/a$')
    ax_rprof.legend()

    ## Fix y range if bool_fixed_y_range is True
    if bool_fixed_y_range:
        ax_rprof.set_ylim(bottom = 0.75*np.amin(p_ma), top = 0.75*np.amax(p_ma))

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

interact(update_plot, 
        N=widgets.IntSlider(min=1, max=100, step=1,value=1),
        dpi_adjust=widgets.FloatSlider(min=30, max=150, step=1,value=dpi_custom),
        param = widgets.Text(value='Er',placeholder='Type something',description='Select parameter:',disabled=False),
        q05_bool    = widgets.Checkbox(value=True,description='q05'),
        q1_bool     = widgets.Checkbox(value=False,description='q1'),
        q15_bool    = widgets.Checkbox(value=True,description='q15'),
        q05n0_bool  = widgets.Checkbox(value=False,description='q05n0'),
        q15n0_bool  = widgets.Checkbox(value=False,description='q15n0'),
        q05new_bool = widgets.Checkbox(value=False,description='q05new'),
        q05br_bool = widgets.Checkbox(value=False,description='q05br'),
        q075br_bool = widgets.Checkbox(value=False,description='q075br'),
        q15br_bool = widgets.Checkbox(value=False,description='q15br'),
        bool_fixed_y_range = widgets.Checkbox(value=False,description='Fix y range'),
        it_play = widgets.Play(value=30,min=0,max=len(qprof_05_br['time']),step=1,interval=150,description="Press play",disabled=False)
        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1, description='N', min=1), FloatSlider(value=100.0, description='dpi_ad…

<function __main__.update_plot(N, dpi_adjust, param, q05_bool, q1_bool, q15_bool, q05n0_bool, q15n0_bool, q05new_bool, q05br_bool, q075br_bool, q15br_bool, it_play, bool_fixed_y_range)>

## <font color='xkcd:sage'>**Exploration**</font> - Profils radiaux, avec moyenne glissante, de quantités choisies

In [ ]:
q_ar = [qprof_05,qprof_15]
param_ar = ['Phi00','Phi01','dPhi01dr','Er','VP','dErdr','RSpol','RSphi','dRSphidr','VT']

param_ar = ['VP']


for ip,p in enumerate(param_ar):
    
    def update_plot(N):
        plt.close('all')
    
        fig = plt.figure(figsize=(xfigsize,6))
        fig.suptitle(p)
        ax1 = fig.add_subplot(111)
        size_window = N * 250 * 1.3 / len(qprof_05['rg']) 
        ax1.set_title(r'Approximative window size for avg : %.1f $\rho_i$' % size_window )
        for iq,q in enumerate(q_ar):

            irhomin = (np.abs(q['rho'] - 0. )).argmin()
            irhomax = (np.abs(q['rho'] - 1. )).argmin()

            itmin = (np.abs(q['time'] - 100e3 )).argmin()
            itmax = (np.abs(q['time'] - 145e3 )).argmin()

            ## Plot raw
            ax1.plot(q['rho'][irhomin:irhomax], np.mean(q[p][itmin:itmax,irhomin:irhomax],axis=0),c=q['c'], ls='--')
            
            ## Plot coarse-grained
            p_ma = uniform_filter1d(uniform_filter1d(np.mean(q[p][itmin:itmax,irhomin:irhomax],axis=0),size=N),size=N)

            #p_ga = gaussian_filter1d(np.mean(q[p][itmin:itmax,:],axis=0),sigma=Ng)[irhomin:irhomax]

            ax1.plot(q['rho'][irhomin:irhomax], p_ma, label=q['title'],lw=3,c=q['c'], ls='-')

            ax1.axhline(y=0,c='k',ls='--')

        #ax1.plot(q['rho'][40:40+Ng],np.ones(Ng)*-0.0,c='k')
        ax1.plot(q['rho'][10:10+N],np.ones(N)*np.amax(p_ma)/2,c='xkcd:battleship grey',lw=5)

        ax1.legend()
        ax1.set_xlabel(r'$r/a$')
    
    interact(update_plot, N=(1, 100, 1), value=60);

interactive(children=(IntSlider(value=50, description='N', min=1), Output()), _dom_classes=('widget-interact',…

## <font color='xkcd:orange'>**(OLD)**</font> <font color='xkcd:sage'>**Exploration**</font> - Evolution spatio-temporelle de quantités choisies

In [ ]:
plt.close('all')
import matplotlib.colors as colors

param_ar = ['VT','VP','Er','dErdr','RSpol','RSphi','Phi01','dPhi01dr']

q_ar = [qprof_05,qprof_15,qprof_05_n0,qprof_15_n0]
param_ar = ['Er']

symetrical_cmap = 0
for ip,p in enumerate(param_ar):
    fig = plt.figure(figsize=(xfigsize,8))
    fig.suptitle(p)
    
    if symetrical_cmap:
        vmin = 0.4*min( abs( np.min(qprof_1[p])) , abs( np.max(qprof_1[p])) )
        vmax = -vmin
    else:
        vmin = 0.4*np.min(qprof_15[p])
        vmax = 0.4*np.max(qprof_15[p])
    
    for iq,q in enumerate(q_ar):
        irhomin = (np.abs(q['rho'] - 0.0 )).argmin()
        irhomax = (np.abs(q['rho'] - 1.1 )).argmin()
        
        itmin = (np.abs(q['time'] - 0e3  )).argmin()
        itmax = (np.abs(q['time'] - 105e3 )).argmin()
        
        #ax = fig.add_subplot(2,len(q_ar)//2,iq+1)
        ax = fig.add_subplot(1,len(q_ar),iq+1)
        ax.set_title(q['title'])
        plt.pcolormesh(q['rho'][irhomin:irhomax],q['time'][itmin:itmax],q[p][itmin:itmax,irhomin:irhomax],vmin=vmin,vmax=vmax)
        #plt.pcolormesh(q['rho'][irhomin:irhomax],q['time'][itmin:itmax],np.abs(q[p][itmin:itmax,irhomin:irhomax]),norm=colors.LogNorm(vmin=1e-4,vmax=1e-3))
        plt.colorbar()
        fig.tight_layout()
        ax.set_xlabel(r'$r/a$')
        ax.set_ylabel(r'time')
        
        ax.axvline(x=1,c='k',ls='--')
        for rline in [0.8,0.6,0.4,0.2]:ax.axvline(x=rline,c='k',ls='--')
        ax.set_xticks([1.0,0.8,0.6,0.4,0.2])
        
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## <font color='xkcd:orange'>**(OLD)**</font> <font color='xkcd:sage'>**Exploration**</font> - Evolution temporelle de quantités choisies

In [ ]:
plt.close('all')

q_ar = [qprof_05_n0,qprof_15_n0]

param_ar = ['VP','Phi00','Er','dErdr','RSpol','RSphi','dRSphidr','VT']

param_ar = ['dRSpoldr']

param_ar = ['VP']

for ip,p in enumerate(param_ar):
    fig = plt.figure(figsize=(6,4))
    fig.suptitle(p)
    ax1 = fig.add_subplot(111)
    for iq,q in enumerate(q_ar):
        
        irhomin = (np.abs(q['rho'] - 0.49 )).argmin()
        irhomax = (np.abs(q['rho'] - 0.51 )).argmin()

        ax1.axhline(y=0,c='k',ls='--')
        
        ax1.set_title(r'avg $%.2f < \rho < %.2f$' % (q['rho'][irhomin],q['rho'][irhomax]))

        ax1.plot(q['time'][:-1], np.mean(q[p][:,irhomin:irhomax],axis=1), c=q['c'],label=q['title'],ls=q['ls'])
        ax1.set_xlabel(r'$\omega_{c0} t$')
        ax1.ticklabel_format(axis='x',scilimits=(3,3),useMathText=True,useOffset=1)
        ax1.legend()
    #ax1.set_xlim(right=70e3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## <font color='xkcd:sage'>**Exploration**</font> - Etude complète d'une quantité donnée dans l'espace réel

In [4]:
plt.close('all')

def update_plot(N,param,float_range_time,float_range_rho,
               q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool,
               colormap_range,dpi_adjust, symetrical_cmap,
               q05br_bool,q075br_bool, q15br_bool):
    
    q_ar = [qprof for qprof, qprof_bool in zip(all_sim, [q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool,q05br_bool,q075br_bool,q15br_bool]) if qprof_bool]
    
    tmin_avg,tmax_avg = float_range_time
    rhomin_avg,rhomax_avg = float_range_rho

    fig = plt.figure(figsize=(16,10))

    ## Set dpi of figure
    fig.set_dpi(dpi_adjust)

    ## Clear previous plot
    for axes in fig.axes: axes.clear()
    
    fig.suptitle(r'Study of %s' %param)

    Pmap = uniform_filter1d(uniform_filter1d(qprof_05[param],size=N,axis=1),size=N,axis=1)
    if symetrical_cmap:
        vmin = colormap_range*min( abs( np.min(Pmap) ), abs( np.max(Pmap) ) ) 
        vmax = -vmin
    else:
        vmin = colormap_range*np.min(Pmap) 
        vmax = colormap_range*np.max(Pmap) 

    gs = gridspec.GridSpec(3, 2*len(q_ar))
    
    for iq,q in enumerate(q_ar):

        irhomin = (np.abs(q['rho'] - 0.0 )).argmin()
        irhomax = (np.abs(q['rho'] - 0.9 )).argmin()

        itmin = (np.abs(q['time'] - 0e3  )).argmin()
        itmax = (np.abs(q['time'] - 150e3 )).argmin()

        p_ma = uniform_filter1d(uniform_filter1d(q[param],size=N,axis=1),size=N,axis=1)
        
        ## Figure 1
        ax_map = fig.add_subplot(gs[0:2, 2*iq:2*iq+2]) 

        ax_map.set_title(q['title'])
        p=ax_map.pcolormesh(q['rho'][irhomin:irhomax],q['time'][itmin:itmax],p_ma[itmin:itmax,irhomin:irhomax],vmin=vmin,vmax=vmax)
        fig.colorbar(p,ax=ax_map)
        fig.tight_layout()
        ax_map.set_xlabel(r'$r/a$')
        ax_map.set_ylabel(r'Time $[\omega_{c0}^{-1}]$')

        ax_map.axvline(x=1,c='k',ls='--')
        for rline in [0.8,0.6,0.4,0.2]:ax_map.axvline(x=rline,c='k',ls='--')
        ax_map.set_xticks([1.0,0.8,0.6,0.4,0.2])
        
        ax_map.axvline(x=rhomin_avg,c='xkcd:purple',ls='-',lw=3); ax_map.axvline(x=rhomax_avg,c='xkcd:purple',ls='-',lw=3)
        ax_map.axhline(y=tmin_avg,c='r',ls='-',lw=3); ax_map.axhline(y=tmax_avg,c='r',ls='-',lw=3)
        
        ax_map.plot(q['rho'][10:10+N],np.ones(N)*ax_map.get_ylim()[1]/4,c='xkcd:battleship grey',lw=5) #Visualize the sliding average window

        ##Figure 2
        itmin_avg = (np.abs(q['time'] - tmin_avg )).argmin()
        itmax_avg = (np.abs(q['time'] - tmax_avg )).argmin()

        ax_rprof = fig.add_subplot(gs[2:, 0:len(q_ar)]) 
        ax_rprof.set_title(r'Radial profile (avg %.0f < t$[\omega_{c0}^{-1}]$ < %.0f)' % (tmin_avg , tmax_avg) )
        ax_rprof.plot(q['rho'][irhomin:irhomax], np.mean( p_ma[itmin_avg:itmax_avg,irhomin:irhomax],axis=0),ls=q['ls'],c=q['c'],label=q['title'])
        ax_rprof.set_xlabel(r'$r/a$')
        ax_rprof.axhline(y=0,c='k',ls='--')
        ax_rprof.legend()
        
        irhomin_avg = (np.abs(q['rho'] - rhomin_avg )).argmin()
        irhomax_avg = (np.abs(q['rho'] - rhomax_avg )).argmin()

        ax_tevol = fig.add_subplot(gs[2:, len(q_ar):]) 
        ax_tevol.set_title(r'Temporal evolution (avg %.2f < r/a < %.2f)' % (rhomin_avg , rhomax_avg) )
        ax_tevol.plot(q['time'][itmin:itmax], np.mean(p_ma[itmin:itmax,irhomin_avg:irhomax_avg],axis=1),ls=q['ls'],c=q['c'],label=q['title'])
        ax_tevol.set_xlabel(r'Time $[\omega_{c0}^{-1}]$')
        ax_tevol.axhline(y=0,c='k',ls='--')
        ax_tevol.legend()

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    
float_range_time_slider = widgets.FloatRangeSlider(value=[100e3, 145e3], min=0, max=150e3, step=1000, description='Time range', layout=Layout(width='50%'), style = {'description_width': 'initial'})
float_range_rho_slider = widgets.FloatRangeSlider(value=[0.8, 0.9], min=0, max=1.2, step=0.05, description='Normalized radius range', layout=Layout(width='50%'), style = {'description_width': 'initial'})
    
interact(update_plot, 
         N=widgets.IntSlider(min=1, max=100, step=1,value=1),
         float_range_time=float_range_time_slider,
         float_range_rho=float_range_rho_slider,
         colormap_range=widgets.FloatSlider(min=0.1, max=2, step=0.01,value=0.8),
         symetrical_cmap = widgets.Checkbox(value=False,description='symetrical_cmap'),
         dpi_adjust=widgets.FloatSlider(min=30, max=100, step=1,value=dpi_custom),
         param = widgets.Text(value='Er',placeholder='Type something',description='Select parameter:',disabled=False),
         q05_bool    = widgets.Checkbox(value=True,description='q05'),
         q1_bool     = widgets.Checkbox(value=False,description='q1'),
         q15_bool    = widgets.Checkbox(value=True,description='q15'),
         q05n0_bool  = widgets.Checkbox(value=False,description='q05n0'),
         q15n0_bool  = widgets.Checkbox(value=False,description='q15n0'),
         q05new_bool = widgets.Checkbox(value=False,description='q05new'),
         q05br_bool = widgets.Checkbox(value=False,description='q05br'),
         q075br_bool = widgets.Checkbox(value=False,description='q075br'),
         q15br_bool = widgets.Checkbox(value=False,description='q15br')
        )

interactive(children=(IntSlider(value=1, description='N', min=1), Text(value='Er', description='Select paramet…

<function __main__.update_plot(N, param, float_range_time, float_range_rho, q05_bool, q1_bool, q15_bool, q05n0_bool, q15n0_bool, q05new_bool, colormap_range, dpi_adjust, symetrical_cmap, q05br_bool, q075br_bool, q15br_bool)>

## <font color='xkcd:sage'>**Exploration**</font> - Etude dans l'espace des fréquences
<font color='red'> /!\ </font> La fréquence théorique des GAMs est calculée comme $f_{theo} = \frac{2}{R_0} \sqrt{ \frac{5}{3} \frac{T}{m}  \left(1 + \frac{1}{2q^2} \right)} \color{red}{\frac{1}{2 \pi}}$ 
En effet, je suspecte que la formule théorique donne la pulsation que tout le monde appelle fréquence par abus de language

<font color='red'> /!\ </font> Les spectres sont calculés avec des moyennes radiales, de cette façon :
- [ ] $\huge TF[\langle s(r,t) \rangle_{r1}^{r2}] $
- [x] $\huge \langle  TF[ s(r,t) ] \rangle_{r1}^{r2}$

PS: les deux sont définis dans le code ci-dessous en tant que "fspectra_1" et "fspectra_2" respectivement.

In [7]:
plt.close('all')
import matplotlib.colors as colors
from scipy.signal import welch

symetrical_cmap = 0

def update_plot(N,colormap_range,dpi_adjust,param,float_range_time,float_range_rho,
               q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool,q05br_bool,
               DFTradial_avg1_bool, DFTradial_avg2_bool, show_omega_theo):
    tmin_avg,tmax_avg = float_range_time
    rhomin_avg,rhomax_avg = float_range_rho
    
    q_ar = [qprof for qprof, qprof_bool in zip(all_sim, [q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool,q05br_bool]) if qprof_bool]
    
    plt.close('all')
    
    fig = plt.figure(figsize=(16,10), dpi=dpi_adjust)
    fig.suptitle(r'Study of %s' %param)

    Pmap = uniform_filter1d(uniform_filter1d(qprof_05[param],size=N,axis=1),size=N,axis=1)
    if symetrical_cmap:
        vmin = colormap_range*min( abs( np.min(Pmap) ), abs( np.max(Pmap) ) ) 
        vmax = -vmin
    else:
        vmin = colormap_range*np.min(Pmap) 
        vmax = colormap_range*np.max(Pmap) 

    gs = gridspec.GridSpec(5, 2*len(q_ar))
    
    for iq,q in enumerate(q_ar):

        irhomin = (np.abs(q['rho'] - 0.0 )).argmin()
        irhomax = (np.abs(q['rho'] - 1.1 )).argmin()

        itmin = (np.abs(q['time'] - 0e3  )).argmin()
        itmax = (np.abs(q['time'] - 90e3 )).argmin()

        p_ma = uniform_filter1d(uniform_filter1d(q[param],size=N,axis=1),size=N,axis=1)
        
        ## Figure 1
        ax_map = fig.add_subplot(gs[0:3, 2*iq:2*iq+2]) 

        ax_map.set_title(q['title'])
        p=ax_map.pcolormesh(q['rho'][irhomin:irhomax],q['time'][itmin:itmax],p_ma[itmin:itmax,irhomin:irhomax],vmin=vmin,vmax=vmax)
        fig.colorbar(p,ax=ax_map)
        fig.tight_layout()
        ax_map.set_xlabel(r'$r/a$')
        ax_map.set_ylabel(r'Time $[\omega_{c0}^{-1}]$')

        ax_map.axvline(x=1,c='k',ls='--')
        for rline in [0.8,0.6,0.4,0.2]:ax_map.axvline(x=rline,c='k',ls='--')
        ax_map.set_xticks([1.0,0.8,0.6,0.4,0.2])
        
        ax_map.axvline(x=rhomin_avg,c='g',ls='-')
        ax_map.axvline(x=rhomax_avg,c='g',ls='-')
        
        ax_map.axhline(y=tmin_avg,c='r',ls='-')
        ax_map.axhline(y=tmax_avg,c='r',ls='-')
        
        ax_map.plot(q['rho'][10:10+N],np.ones(N)*ax_map.get_ylim()[1]/4,c='xkcd:battleship grey',lw=5) #Visualisation de la fenêtre de la moyenne glissante

        
        ##Figure 2
        itmin_avg = (np.abs(q['time'] - tmin_avg )).argmin()
        itmax_avg = (np.abs(q['time'] - tmax_avg )).argmin()

        irhomin_avg = (np.abs(q['rho'] - rhomin_avg )).argmin()
        irhomax_avg = (np.abs(q['rho'] - rhomax_avg )).argmin()
        
        #Perform fft
        dt_diag = q['time'][-1]-q['time'][-2]
        fspectra_1 = np.fft.rfft(np.mean(p_ma[itmin_avg:itmax_avg,irhomin_avg:irhomax_avg],axis=1), axis=0) / (itmax_avg-itmin_avg)
        fspectra_2 = np.fft.rfft(p_ma[itmin_avg:itmax_avg,:], axis=0) / (itmin_avg-itmax_avg)
        w = np.fft.rfftfreq(itmax_avg-itmin_avg, d=dt_diag)
        
        #Try with Welsh
        #f_welsh, spectra_welsh = welch(np.mean(p_ma[itmin_avg:itmax_avg,irhomin_avg:irhomax_avg],axis=1), fs=1/dt_diag, nperseg=100, window='hamming', axis=0)

        #Comparaison avec la fonction implémentée dans GYSELA
        #fspectra_GYS, w_GYS = Fourier1D( p_ma[itmin_avg:itmax_avg,:], q['time'][itmin_avg:itmax_avg],axis=0)
        
        ax_fspectra = fig.add_subplot(gs[3:, 0:len(q_ar)]) 
        ax_fspectra.set_title(r'Fourier amplitude (avg %.2f < r/a < %.2f)' % (rhomin_avg , rhomax_avg) )
        if DFTradial_avg1_bool:ax_fspectra.semilogy(w,np.abs(fspectra_1),c=q['c'],label=q['title'],ls='--') #Pour fspectra_1
        #ax_fspectra.semilogy(f_welsh,np.abs(spectra_welsh)/(itmax_avg-itmin_avg),c=q['c'],label=q['title'],ls='--') #Pour Welsh
        if DFTradial_avg2_bool:ax_fspectra.semilogy(w,np.mean(np.abs(fspectra_2[:,irhomin_avg:irhomax_avg]),axis=1),ls='-',c=q['c'],label=q['title']) #Pour fspectra_2
        #ax_fspectra.semilogy(w_GYS/(2*np.pi),np.mean(np.abs(fspectra_GYS[:,irhomin_avg:irhomax_avg]),axis=1),ls='--',c=q['c'],label=q['title']) #Pour comparer avec la fonction native dans les diags GYSELA
        ax_fspectra.set_xlabel(r'$\omega [\omega_{c0}]$')
        ax_fspectra.legend(fontsize=14)
        
        #Fréquence GAM théorique
        if show_omega_theo:
            omega_theo = np.mean( np.mean( (2*np.sqrt(q['T'] * (5/3) *(1 + (1/(2*q['q']**2) ) )) /800)[itmin_avg:itmax_avg] ,axis=0)[irhomax_avg-1:irhomax_avg] ) / (2 * np.pi)
            ax_fspectra.axvline(x=omega_theo,c=q['c'],ls='--')
        
        ax_fphase = fig.add_subplot(gs[3:, len(q_ar):]) 
        ax_fphase.set_title(r'Fourier phase (avg %.2f < r/a < %.2f)' % (rhomin_avg , rhomax_avg) )
        if DFTradial_avg1_bool:ax_fphase.plot(np.sort(w), np.angle(fspectra_1),ls='--',c=q['c'],label=q['title']) #Pour fspectra_1
        if DFTradial_avg2_bool:ax_fphase.plot(np.sort(w), np.mean(np.angle(fspectra_2[:,irhomin_avg:irhomax_avg]),axis=1),ls='-',c=q['c'],label=q['title']) #Pour fspectra_2
        ax_fphase.set_xlabel(r'$\omega [\omega_{c0}]$')
        ax_fphase.axhline(y=0,ls='--',c='k')
        ax_fphase.legend(fontsize=16)

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

float_range_time_slider = widgets.FloatRangeSlider(value=[50e3, 90e3], min=0, max=150e3, step=1000, description='Time range', layout=Layout(width='50%'), style = {'description_width': 'initial'})
    
float_range_rho_slider = widgets.FloatRangeSlider(value=[0.84, 0.86], min=0, max=1.2, step=0.01, description='Normalized radius range', layout=Layout(width='50%'), style = {'description_width': 'initial'})
    
interact(update_plot, 
         N=widgets.IntSlider(min=1, max=100, step=1,value=1),
         float_range_time=float_range_time_slider,
         float_range_rho=float_range_rho_slider,
         colormap_range=widgets.FloatSlider(min=0.1, max=2, step=0.05,value=0.8),
         dpi_adjust=widgets.FloatSlider(min=30, max=100, step=1,value=dpi_custom),
         param = widgets.Text(value='Ptransfert',placeholder='Type something',description='Select parameter:',disabled=False),
         q05_bool    = widgets.Checkbox(value=True,description='q05'),
         q1_bool     = widgets.Checkbox(value=False,description='q1'),
         q15_bool    = widgets.Checkbox(value=True,description='q15'),
         q05n0_bool  = widgets.Checkbox(value=False,description='q05n0'),
         q15n0_bool  = widgets.Checkbox(value=False,description='q15n0'),
         q05new_bool = widgets.Checkbox(value=False,description='q05new'),
         q05br_bool  = widgets.Checkbox(value=False,description='q05br'),
         DFTradial_avg1_bool = widgets.Checkbox(value=False,description='Display fspectra_1'),
         DFTradial_avg2_bool = widgets.Checkbox(value=True,description='Display fspectra_2'),
         show_omega_theo = widgets.Checkbox(value=False,description=r'Display $\omega_{theo}$')
        )

interactive(children=(IntSlider(value=1, description='N', min=1), FloatSlider(value=0.8, description='colormap…

<function __main__.update_plot(N, colormap_range, dpi_adjust, param, float_range_time, float_range_rho, q05_bool, q1_bool, q15_bool, q05n0_bool, q15n0_bool, q05new_bool, q05br_bool, DFTradial_avg1_bool, DFTradial_avg2_bool, show_omega_theo)>

## <font color='xkcd:sage'>**Exploration**</font> - Densité spectrale de puissance

La DPS est calculée comme $ PSD(x) = 2 \frac{ |X(\omega)|^2 }{ N } \Delta T $ où $N$ est le nombre d'échantillons temporels (le facteur 2 vient du fait qu'on considère le nombre de points $N_f=N/2$ dans le domaine spectral).

Ce calcul donne exactement le même résultat que la fonction *periodogram* de *scipy.signal* ... sauf pour la fréquence nulle.

In [ ]:
plt.close('all')
import matplotlib.colors as colors
import scipy.signal
from scipy.signal import welch


symetrical_cmap = 0
def update_plot(N,colormap_range,dpi_adjust,param,float_range_time,float_range_rho,
               q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool,
               q05br_bool,q075br_bool,q15br_bool,
               mode):#Nwelsh, overlap_factor):
    
    tmin_avg,tmax_avg = float_range_time
    rhomin_avg,rhomax_avg = float_range_rho
    
    q_ar = [qprof for qprof, qprof_bool in zip(all_sim, [q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool,q05br_bool,q075br_bool,q15br_bool]) if qprof_bool]
    
    plt.close('all')
    
    fig = plt.figure(figsize=(16,10), dpi=dpi_adjust)
    fig.suptitle(r'Study of %s' %param)

    Pmap = uniform_filter1d(uniform_filter1d(qprof_05[param],size=N,axis=1),size=N,axis=1)
    if symetrical_cmap:
        vmin = colormap_range*min( abs( np.min(Pmap) ), abs( np.max(Pmap) ) ) 
        vmax = -vmin
    else:
        vmin = colormap_range*np.min(Pmap) 
        vmax = colormap_range*np.max(Pmap) 

    gs = gridspec.GridSpec(5, 2*len(q_ar))
    
    for iq,q in enumerate(q_ar):

        irhomin = (np.abs(q['rho'] - 0.0 )).argmin()
        irhomax = (np.abs(q['rho'] - 1.1 )).argmin()

        itmin = (np.abs(q['time'] - 0e3  )).argmin()
        itmax = (np.abs(q['time'] - 145e3 )).argmin()

        p_ma = uniform_filter1d(uniform_filter1d(q[param],size=N,axis=1),size=N,axis=1)
        
        ## Figure 1
        ax_map = fig.add_subplot(gs[0:3, 2*iq:2*iq+2]) 

        ax_map.set_title(q['title'])
        p=ax_map.pcolormesh(q['rho'][irhomin:irhomax],q['time'][itmin:itmax],p_ma[itmin:itmax,irhomin:irhomax],vmin=vmin,vmax=vmax)
        fig.colorbar(p,ax=ax_map)
        fig.tight_layout()
        ax_map.set_xlabel(r'$r/a$')
        ax_map.set_ylabel(r'Time $[\omega_{c0}^{-1}]$')

        ax_map.axvline(x=1,c='k',ls='--')
        for rline in [0.8,0.6,0.4,0.2]:ax_map.axvline(x=rline,c='k',ls='--')
        ax_map.set_xticks([1.0,0.8,0.6,0.4,0.2])
        
        ax_map.axvline(x=rhomin_avg,c='g',ls='-')
        ax_map.axvline(x=rhomax_avg,c='g',ls='-')
        
        ax_map.axhline(y=tmin_avg,c='r',ls='-')
        ax_map.axhline(y=tmax_avg,c='r',ls='-')
        
        ax_map.plot(q['rho'][10:10+N],np.ones(N)*ax_map.get_ylim()[1]/4,c='xkcd:battleship grey',lw=5) #Visualisation de la fenêtre de la moyenne glissante

        
        ##Figure 2
        itmin_avg = (np.abs(q['time'] - tmin_avg )).argmin()
        itmax_avg = (np.abs(q['time'] - tmax_avg )).argmin()

        irhomin_avg = (np.abs(q['rho'] - rhomin_avg )).argmin()
        irhomax_avg = (np.abs(q['rho'] - rhomax_avg )).argmin()
        
        #Perform fft
        dt_diag = q['time'][-1]-q['time'][-2]
        fspectra = np.fft.rfft(p_ma[itmin_avg:itmax_avg,:], axis=0) / (itmax_avg-itmin_avg)
        w = np.fft.rfftfreq(itmax_avg-itmin_avg, d=dt_diag)
        
        f_DSP, DSP = scipy.signal.periodogram(p_ma[itmin_avg:itmax_avg,:], 1/dt_diag, scaling='density', axis=0)
        
        #Welsh
        #print(Nwelsh)
        #print(Nwelsh*overlap_factor)
        #f_welsh, spectra_welsh = welch(p_ma[itmin_avg:itmax_avg,:], fs=1/dt_diag, nperseg=Nwelsh, noverlap=int(Nwelsh*overlap_factor), window='hann', scaling='density', axis=0)
        
        ax_fspectra = fig.add_subplot(gs[3:, 0:len(q_ar)]) 
        ax_fspectra.set_title(r'Fourier amplitude (avg %.2f < r/a < %.2f)' % (rhomin_avg , rhomax_avg) )
        #ax_fspectra.semilogy(f_DSP,np.mean(DSP[:,irhomin_avg:irhomax_avg],axis=1),c=q['c'],label=q['title'],ls='-') #nécéssite peut être une normalisation par (itmax_avg-itmin_avg)
        ax_fspectra.semilogy(w,2*np.mean(np.abs(fspectra[:,irhomin_avg:irhomax_avg])**2 * ((itmax_avg-itmin_avg)*dt_diag),axis=1),c=q['c'],label=q['title'],ls='-')
        #ax_fspectra.semilogy(f_welsh,np.mean(np.abs(spectra_welsh[:,irhomin_avg:irhomax_avg]),axis=1),c=q['c'],label=q['title'],ls='--') #Pour Welsh, nécéssite peut être une normalisation par (itmax_avg-itmin_avg)
        ax_fspectra.set_xlabel(r'$\omega [\omega_{c0}]$')
        ax_fspectra.legend(fontsize=14)
        ax_fspectra.axvline(x=w[mode], ls='--', c='g')

        irholimmin = (np.abs(q['rho'] - 0.5 )).argmin()
        irholimmax = (np.abs(q['rho'] - 1 )).argmin()
        ax_rprof = fig.add_subplot(gs[3:, len(q_ar):]) 
        ax_rprof.set_title(r'Profile of $\omega = %.2e [\omega_{c0}^1]$ component' % w[mode])
        ax_rprof.semilogy(q['rho'][irholimmin:irholimmax], 
                      2*np.abs(fspectra[mode,irholimmin:irholimmax])**2 * ((itmax_avg-itmin_avg)*dt_diag), c=q['c']
                     )
        
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

float_range_time_slider = widgets.FloatRangeSlider(value=[100e3, 145e3], min=0, max=150e3, step=1000, description='Time range', layout=Layout(width='50%'), style = {'description_width': 'initial'})

float_range_rho_slider = widgets.FloatRangeSlider(value=[0.84, 0.86], min=0, max=1.2, step=0.01, description='Normalized radius range', layout=Layout(width='50%'), style = {'description_width': 'initial'})
    
interact(update_plot, 
         N=widgets.IntSlider(min=1, max=100, step=1,value=1),
         mode=widgets.IntSlider(min=0, max=50, step=1,value=0),
         #Nwelsh=widgets.IntSlider(min=1, max=1000, step=1,value=2),
         overlap_factor=widgets.FloatSlider(min=0, max=1, step=0.05,value=0.8),
         float_range_time=float_range_time_slider,
         float_range_rho=float_range_rho_slider,
         colormap_range=widgets.FloatSlider(min=0.1, max=2, step=0.05,value=0.8),
         dpi_adjust=widgets.FloatSlider(min=30, max=100, step=1,value=dpi_custom),
         param = widgets.Text(value='Ptransfert',placeholder='Type something',description='Select parameter:',disabled=False),
         q05_bool    = widgets.Checkbox(value=True,description='q05'),
         q1_bool     = widgets.Checkbox(value=False,description='q1'),
         q15_bool    = widgets.Checkbox(value=True,description='q15'),
         q05n0_bool  = widgets.Checkbox(value=False,description='q05n0'),
         q15n0_bool  = widgets.Checkbox(value=False,description='q15n0'),
         q05new_bool = widgets.Checkbox(value=False,description='q05new'),
         q05br_bool  = widgets.Checkbox(value=False,description='q05br'),
         q075br_bool = widgets.Checkbox(value=False,description='q075br'),
         q15br_bool  = widgets.Checkbox(value=False,description='q15br'),
         show_omega_theo = widgets.Checkbox(value=False,description=r'Display $\omega_{theo}$')
        )

interactive(children=(IntSlider(value=1, description='N', min=1), FloatSlider(value=0.8, description='colormap…

<function __main__.update_plot(N, colormap_range, dpi_adjust, param, float_range_time, float_range_rho, q05_bool, q1_bool, q15_bool, q05n0_bool, q15n0_bool, q05new_bool, q05br_bool, q075br_bool, q15br_bool, mode)>

No such comm: 0b63101220b04852a869a86b2f8e04bb
No such comm: 0b63101220b04852a869a86b2f8e04bb
No such comm: 0b63101220b04852a869a86b2f8e04bb
No such comm: 0b63101220b04852a869a86b2f8e04bb


No such comm: dc36c2fe6b2e4dd2b167a968d21aa843
No such comm: dc36c2fe6b2e4dd2b167a968d21aa843
No such comm: dc36c2fe6b2e4dd2b167a968d21aa843
No such comm: dc36c2fe6b2e4dd2b167a968d21aa843
No such comm: b43c67534c1242b895756f6043fe87fd
No such comm: b43c67534c1242b895756f6043fe87fd
No such comm: b43c67534c1242b895756f6043fe87fd
No such comm: b43c67534c1242b895756f6043fe87fd


## <font color='xkcd:sage'>**Exploration**</font> - Cross corrélation en temps
ps: un time-lag $\tau$ négatif veut dire que "signal2" précède "signal1" de $\tau$

In [22]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def extract_fluctuations(signal2D):
    return signal2D - np.mean(signal2D,axis=0)

def update_plot(N1,N2,float_range_rho,dpi_adjust,float_range_time,
               q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool,qgdp_bool,
               signal1, signal2):
    
    q_ar = []
    if q05_bool    : q_ar = np.append(q_ar,qprof_05)
    if q1_bool     : q_ar = np.append(q_ar,qprof_1)
    if q15_bool    : q_ar = np.append(q_ar,qprof_15)
    if q05n0_bool  : q_ar = np.append(q_ar,qprof_05_n0)
    if q15n0_bool  : q_ar = np.append(q_ar,qprof_15_n0)
    if q05new_bool : q_ar = np.append(q_ar,qprof_05new)
    if qgdp_bool   : q_ar = np.append(q_ar,qprof_gdp)
    
    tmin_avg,tmax_avg = float_range_time
    
    plt.close('all')
    
    fig = plt.figure(figsize=(11*len(q_ar),16), dpi=dpi_adjust)
    fig.suptitle('Cross correlation between %s and %s' %(signal1,signal2))
    
    gs = gridspec.GridSpec(3,len(q_ar))
    
    ###############################
    #Figure 3 layout - Cross correlation at a given radius
    ###############################
    ax_cc1D = fig.add_subplot(gs[2, :]) 
    ax_cc1D.set_ylim(-1.1,1.1)
    ax_cc1D.axhline(y=0,c='k',ls='--'); ax_cc1D.axhline(y=1,c='gray',ls='--'); ax_cc1D.axhline(y=-1,c='gray',ls='--')
    ax_cc1D.axvline(x=0,c='k',ls='--')
    ax_cc1D.set_xlabel(r'Time lag $[\omega_{c0}^{-1}]$')
    
    for iq,q in enumerate(q_ar):

        itmin = (np.abs(q['time'] - tmin_avg )).argmin()//3*3
        itmax = (np.abs(q['time'] - tmax_avg )).argmin()//3*3

        irhomin = (np.abs(q['rho'] - 0.0 )).argmin()
        irhomax = (np.abs(q['rho'] - 1.1 )).argmin()

        itlim = itmax-itmin
        itau_range = np.r_[-itlim//3:itlim//3]

        tau_range = itau_range * (q['time'][1]-q['time'][0])

        it_CC_range = np.r_[itlim//3:2*itlim//3]
        Cfg = np.zeros( (irhomax-irhomin, 2*itlim//3 ) )
        
        signal1_ar = uniform_filter1d(uniform_filter1d(q[signal1],size=N1,axis=1),size=N1,axis=1)
        signal2_ar = uniform_filter1d(uniform_filter1d(q[signal2],size=N2,axis=1),size=N2,axis=1)

        f = extract_fluctuations(signal1_ar[itmin:itmax,irhomin:irhomax])
        g = extract_fluctuations(signal2_ar[itmin:itmax,irhomin:irhomax])

        norm = np.sqrt( np.sum(f[it_CC_range,:]**2,axis=0) ) * np.sqrt( np.sum(g[it_CC_range,:]**2,axis=0) )
        for itau in itau_range:
            Cfg[:,itau + itlim//3] = np.sum(f[it_CC_range,:] * g[it_CC_range+itau,:],axis=0)/norm


        rhomin_avg,rhomax_avg = float_range_rho
        irhomin_avg = (np.abs(q['rho'] - rhomin_avg )).argmin()
        irhomax_avg = (np.abs(q['rho'] - rhomax_avg )).argmin()
        ##############################
        #Figure - 2D cross correlation
        ##############################
        ax_map = fig.add_subplot(gs[0, iq]) 
        ax_map.set_title('Cross correlation over radius')
        p=ax_map.pcolormesh(q['rho'][irhomin:irhomax],tau_range,Cfg.T,vmin=-1,vmax=1)
        ax_map.axhline(y=0,c='k',ls='--')
        cax = inset_axes(ax_map, width="30%", height="5%", loc=1, bbox_to_anchor=(0,0.2,1,1), bbox_transform=ax_map.transAxes) 
        fig.colorbar(p,cax=cax,orientation='horizontal')
        ax_map.axvline(x=rhomin_avg,c='g',lw=4)
        ax_map.axvline(x=rhomax_avg,c='g',lw=4)
        ax_map.set_title(r'%s' % q['title'])
        ax_map.set_xlabel(r'$r/a$')
        ax_map.set_ylabel(r'Time lag')
        ax_map.set_xticks([1.0,0.8,0.6,0.4,0.2])
        
        ##############################
        #Figure - Time evolution
        ##############################
        color1 = 'xkcd:green'
        color2 = 'xkcd:purple'
        
        itbeg = (np.abs(q['time'] - 0e3 )).argmin()//3*3
        ax_tevol1 = fig.add_subplot(gs[1, iq]) 
        ax_tevol1.set_title('%s - Time evolution avg on $%.2f < r/a < %.2f$' %(q['title'],rhomin_avg,rhomax_avg))
        ax_tevol1.plot(q['time'][itbeg:],np.mean(q[signal1][itbeg:,irhomin_avg:irhomax_avg],axis=1),c=color1,ls='--')
        ax_tevol1.plot(q['time'][itbeg:],np.mean(signal1_ar[itbeg:,irhomin_avg:irhomax_avg],axis=1),c=color1)
        ax_tevol1.yaxis.label.set_color(color1)
        ax_tevol1.tick_params(axis='y', colors=color1)
        ax_tevol1.set_ylabel(signal1)
        ax_tevol2 = ax_tevol1.twinx()
        ax_tevol2.plot(q['time'][itbeg:],np.mean(q[signal2][itbeg:,irhomin_avg:irhomax_avg],axis=1),c=color2,ls='--')
        ax_tevol2.plot(q['time'][itbeg:],np.mean(signal2_ar[itbeg:,irhomin_avg:irhomax_avg],axis=1),c=color2)
        ax_tevol2.grid(None)
        ax_tevol2.yaxis.label.set_color(color2)
        ax_tevol2.tick_params(axis='y', colors=color2)
        ax_tevol2.set_ylabel(signal2)
        ax_tevol1.axvline(x=tmin_avg,c='k',ls='--'); ax_tevol1.axvline(x=tmax_avg,c='k',ls='--')
        ax_tevol1.set_xlabel(r'Time $[\omega_{c0}^{-1}]$')
        
        ##############################
        #Figure 3 layout - Cross correlation at a given radius
        ##############################
        ax_cc1D.set_title(r'Cross correlation avg on $%.2f < r/a < %.2f$' %(rhomin_avg,rhomax_avg))
        ax_cc1D.plot(tau_range, np.mean(Cfg[irhomin_avg:irhomax_avg,:],axis=0),c=q['c'],lw=4,label=q['title'])
    
    ax_cc1D.legend()
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

float_range_time_slider = widgets.FloatRangeSlider(value=[100e3, 145e3], min=0, max=150e3, step=1000, description='Time range', layout=Layout(width='50%'), style = {'description_width': 'initial'})
float_range_rho_slider = widgets.FloatRangeSlider(value=[0.8, 0.9], min=0, max=1.2, step=0.01, description='Normalized radius range', layout=Layout(width='50%'), style = {'description_width': 'initial'})
    
interact(update_plot, 
         N1=widgets.IntSlider(min=1, max=200, step=1,value=1),
         N2=widgets.IntSlider(min=1, max=200, step=1,value=1),
         float_range_time=float_range_time_slider,
         float_range_rho=float_range_rho_slider,
         dpi_adjust=widgets.FloatSlider(min=30, max=100, step=1,value=dpi_custom),
         q05_bool    = widgets.Checkbox(value=True,description='q05'),
         q1_bool     = widgets.Checkbox(value=False,description='q1'),
         q15_bool    = widgets.Checkbox(value=True,description='q15'),
         q05n0_bool  = widgets.Checkbox(value=False,description='q05n0'),
         q15n0_bool  = widgets.Checkbox(value=False,description='q15n0'),
         q05new_bool = widgets.Checkbox(value=False,description='q05new'),
         qgdp_bool   = widgets.Checkbox(value=False,description='qgdp'),
         signal1 = widgets.Text(value='dRSpoldr',placeholder='Type something',description='Select parameter:',disabled=False),
         signal2 = widgets.Text(value='Er',placeholder='Type something',description='Select parameter:',disabled=False)
        )

interactive(children=(IntSlider(value=1, description='N1', max=200, min=1), IntSlider(value=1, description='N2…

<function __main__.update_plot(N1, N2, float_range_rho, dpi_adjust, float_range_time, q05_bool, q1_bool, q15_bool, q05n0_bool, q15n0_bool, q05new_bool, qgdp_bool, signal1, signal2)>

## <font color='orangered'>**(En cours)**</font> <font color='xkcd:sage'>**Exploration**</font> - Cycle limite

In [85]:
plt.close('all')
import matplotlib.colors as colors
import scipy.signal 

symetrical_cmap = 0
def update_plot(N,colormap_range,dpi_adjust,param,param2,float_range_time,float_range_rho,
               q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool, show_omega_theo):#Nwelsh, overlap_factor):
    
    tmin_avg,tmax_avg = float_range_time
    rhomin_avg,rhomax_avg = float_range_rho
    
    q_ar = []
    if q05_bool    : q_ar = np.append(q_ar,qprof_05)
    if q1_bool     : q_ar = np.append(q_ar,qprof_1)
    if q15_bool    : q_ar = np.append(q_ar,qprof_15)
    if q05n0_bool  : q_ar = np.append(q_ar,qprof_05_n0)
    if q15n0_bool  : q_ar = np.append(q_ar,qprof_15_n0)
    if q05new_bool : q_ar = np.append(q_ar,qprof_05new)
    
    plt.close('all')
    
    fig = plt.figure(figsize=(16,8), dpi=dpi_adjust)

    Pmap = uniform_filter1d(uniform_filter1d(qprof_05[param],size=N,axis=1),size=N,axis=1)
    if symetrical_cmap:
        vmin = colormap_range*min( abs( np.min(Pmap) ), abs( np.max(Pmap) ) ) 
        vmax = -vmin
    else:
        vmin = colormap_range*np.min(Pmap) 
        vmax = colormap_range*np.max(Pmap) 

    gs = gridspec.GridSpec(5, 2*len(q_ar))
    
    for iq,q in enumerate(q_ar):

        irhomin = (np.abs(q['rho'] - rhomin_avg )).argmin()
        irhomax = (np.abs(q['rho'] - rhomax_avg )).argmin()

        itmin = (np.abs(q['time'] - tmin_avg  )).argmin()
        itmax = (np.abs(q['time'] - tmax_avg )).argmin()

        p_ma = uniform_filter1d(uniform_filter1d(q[param],size=N,axis=1),size=N,axis=1)
        
        ## Figure 1
        ax_cl = fig.add_subplot(gs[:, 2*iq:2*iq+2]) 

        ax_cl.set_title('Cycle limite ' + q['title'])
        ax_cl.set_xlabel(param)
        ax_cl.set_ylabel(param2)
        
        x = np.mean(q[param][itmin:itmax,irhomin:irhomax],axis=1)
        y = np.mean(q[param2][itmin:itmax,irhomin:irhomax],axis=1)
        
        ax_cl.plot(x,y,c='k')
        ax_cl.scatter(x,y,
                   c=np.tan( q['time'][itmin:itmax]/q['time'][itmax] ),
                   marker='_', s=80,
                   lw=5, cmap='winter',zorder=10)
        
        ax_cl.autoscale()
        
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

float_range_time_slider = widgets.FloatRangeSlider(value=[100e3, 145e3], min=0, max=150e3, step=1000, description='Time range', layout=Layout(width='50%'), style = {'description_width': 'initial'})
    
float_range_rho_slider = widgets.FloatRangeSlider(value=[0.8, 0.9], min=0, max=1.2, step=0.01, description='Normalized radius range', layout=Layout(width='50%'), style = {'description_width': 'initial'})
    
interact(update_plot, 
         N=widgets.IntSlider(min=1, max=100, step=1,value=1),
         float_range_time=float_range_time_slider,
         float_range_rho=float_range_rho_slider,
         colormap_range=widgets.FloatSlider(min=0.1, max=2, step=0.05,value=0.8),
         dpi_adjust=widgets.FloatSlider(min=30, max=100, step=1,value=dpi_custom),
         param = widgets.Text(value='RSpol',placeholder='Type something',description='Select parameter:',disabled=False),
         param2 = widgets.Text(value='gradT',placeholder='Type something',description='Select parameter:',disabled=False),
         q05_bool    = widgets.Checkbox(value=True,description='q05'),
         q1_bool     = widgets.Checkbox(value=False,description='q1'),
         q15_bool    = widgets.Checkbox(value=True,description='q15'),
         q05n0_bool  = widgets.Checkbox(value=False,description='q05n0'),
         q15n0_bool  = widgets.Checkbox(value=False,description='q15n0'),
         q05new_bool = widgets.Checkbox(value=False,description='q05new'),
         show_omega_theo = widgets.Checkbox(value=False,description=r'Display $\omega_{theo}$')
        )

interactive(children=(IntSlider(value=1, description='N', min=1), FloatSlider(value=0.8, description='colormap…

<function __main__.update_plot(N, colormap_range, dpi_adjust, param, param2, float_range_time, float_range_rho, q05_bool, q1_bool, q15_bool, q05n0_bool, q15n0_bool, q05new_bool, show_omega_theo)>

## <font color='coral'>**Travail en cours**</font> - Analyse des termes dans l'évolution de $E_r$

L'équation de conservation de la vorticité généralisée peut se réécrire sous la forme :

$ \huge \partial_t E_r = \frac{B_{eq}^2}{n_{eq} m} J_r - \frac{1}{2} \frac{1}{n_{eq} e} \partial_t ( \nabla_\perp P_\perp ) $

## <font color='red'>**Résultat**</font> - Profil radial du **facteur de sécurité** $q$

In [6]:
plt.rc('font',size=22)
q_ar = [qprof_05,qprof_1,qprof_15]
q_ar = [qprof_05_br,qprof_075_br,qprof_15_br]

fig = plt.figure(figsize=(8,6))
ax1 = fig.add_subplot(111)

for iq,q in enumerate(q_ar):
    ax1.set_title('Safety factor')
    irho = (np.abs(q['rho'] - 0.95 )).argmin()
    
    ax1.plot(q['rho'], q['q'], label=q['title']+r', $q_{95} = %.1f$' % q['q'][irho],c=q['c'],lw=4)
ax1.legend()
ax1.set_xlabel(r'$r/a$')
ax1.set_ylabel(r'$q$')
ax1.axvline(x=1,ls='--',c='k')
ax1.set_xlim(left=-0.01,right=1.31)

ax1.axhline(y=1,c='k',ls='--')

fig.tight_layout()

#fig.savefig('Figures/Scan_Ip_q_profiles.pdf', format='pdf', bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

FileNotFoundError: [Errno 2] No such file or directory: 'Figures/Scan_Ip_q_profiles.pdf'

## <font color='red'>**Résultat**</font> - Profil radial du **tenseur de Reynolds $\Pi_{r \theta}$** (moyenné en temps **et** coarse-grained)

In [ ]:
q_ar = [qprof_05,qprof_1,qprof_15]

fig = plt.figure(figsize=(7,7))
#fig.suptitle(p)
ax1 = fig.add_subplot(111)

N=60
def u1d(X):
    #return uniform_filter1d(X,size=N)
    return uniform_filter1d(uniform_filter1d(X,size=N),size=N)
    #return uniform_filter1d(uniform_filter1d(uniform_filter1d(X,size=N),size=N),size=N)
    #return uniform_filter1d(uniform_filter1d(uniform_filter1d(uniform_filter1d(X,size=N),size=N),size=N),size=N)

def g1d(X):
    return gaussian_filter1d(X,sigma=N)

for iq,q in enumerate(q_ar):
    irhomin = (np.abs(q['rho'] - 0.1 )).argmin()
    irhomax = (np.abs(q['rho'] - 1.0 )).argmin()

    itmin = (np.abs(q['time'] - 100.0e3 )).argmin()
    itmax = (np.abs(q['time'] - 145.0e3 )).argmin()

    
    ax1.plot(q['rho'][irhomin:irhomax], np.mean(q['RSpol'][itmin:itmax,irhomin:irhomax],axis=0),ls='--',c=q['c'],lw=2)
    ax1.plot(q['rho'][irhomin:irhomax], u1d(np.mean(q['RSpol'][itmin:itmax,irhomin:irhomax],axis=0)),c=q['c'],label=q['title'],lw=4)

    ax1.axhline(y=0,ls='--',c='k')

ax1.set_ylabel(r'$\Pi_{r \theta}$ $[V_{T0}^2]$')

ax1.ticklabel_format(axis='y',scilimits=(-1,2),useMathText=True)

ax1.legend()
ax1.set_xlabel(r'$r/a$')

plt.text(x=0.01, y=0.9, s='Plain: coarse-grained \ndashed: time average only', transform=ax1.transAxes)

fig.tight_layout(rect=[0, 0.03, 1, 0.95])

fig.savefig('Figures/rprof_RSpol_scanq.pdf', format='pdf', bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## <font color='red'>**Résultat**</font> - Profil radial du **coarse-grained $- \nabla \cdot \Pi_{r_\theta}$ avec une fenêtre radiale de $\sim 40 \rho_i$**

In [ ]:
q_ar = [qprof_05,qprof_1,qprof_15]

param_ar = ['dRSpoldr']

N=60
Ng=30

for ip,p in enumerate(param_ar):
    fig = plt.figure(figsize=(xfigsize,6))
    ax1 = fig.add_subplot(111)
    for iq,q in enumerate(q_ar):
        
        irhomin = (np.abs(q['rho'] - 0. )).argmin()
        irhomax = (np.abs(q['rho'] - 1. )).argmin()

        itmin = (np.abs(q['time'] - 100e3 )).argmin()
        itmax = (np.abs(q['time'] - 145e3 )).argmin()
        
        p_ma = uniform_filter1d(uniform_filter1d(np.mean(q[p][itmin:itmax,irhomin:irhomax],axis=0),size=N),size=N)
        
        p_ga = gaussian_filter1d(np.mean(q[p][itmin:itmax,:],axis=0),sigma=Ng)[irhomin:irhomax]
        
        ax1.plot(q['rho'][irhomin:irhomax], -p_ma, label=q['title'],lw=3,c=q['c'], ls='-')
        

        ax1.axhline(y=0,c='k',ls='--')
        

        
    ax1.legend()
    ax1.set_xlabel(r'$r/a$')
    ax1.set_ylabel(r'$-\langle \nabla \cdot \Pi_{r \theta} \rangle_{CG}$')
    
    fig.tight_layout()
    
    fig.savefig('Figures/rprof_divPiCG_scanq.pdf', format='pdf', bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## <font color='red'>**Résultat**</font> - Profil en $q$ du **coarse-grained $- \nabla \cdot \Pi_{r_\theta}$**

In [ ]:
q_ar = [qprof_05,qprof_1,qprof_15]
param_ar = ['Phi00','Phi01','dPhi01dr','Er','VP','dErdr','RSpol','RSphi','dRSphidr','VT']

param_ar = ['dRSpoldr']

N=60
Ng=30

for ip,p in enumerate(param_ar):
    fig = plt.figure(figsize=(xfigsize,6))
    #fig.suptitle(p)
    ax1 = fig.add_subplot(111)
    for iq,q in enumerate(q_ar):
        
        irhomin = (np.abs(q['rho'] - 0.79 )).argmin()
        irhomax = (np.abs(q['rho'] - 0.81 )).argmin()

        itmin = (np.abs(q['time'] - 100e3 )).argmin()
        itmax = (np.abs(q['time'] - 145e3 )).argmin()
        
        p_ma = uniform_filter1d(uniform_filter1d(np.mean(q[p][itmin:itmax,:],axis=0),size=N),size=N)
        
        p_ga = gaussian_filter1d(np.mean(q[p][itmin:itmax,:],axis=0),sigma=Ng)[irhomin:irhomax]
        
        
        ax1.plot(q['q'][irhomin:irhomax], -p_ma[irhomin:irhomax], lw=3,c=q['c'], marker='s',ls='', label=q['title'])
        
        #ax1.plot(q['q'][irhomin:irhomax], 0.3e-7*(-1+q['q'][irhomin:irhomax])**(1/3), c=q['c'])
        
        ax1.axhline(y=0,c='k',ls='--')
        
#     ax1.plot(q['rho'][40:40+Ng],np.ones(Ng)*-0.02,c='k')
#     ax1.plot(q['rho'][100:100+N],np.ones(N)*-0.02,c='k')
        
    
    ax1.set_xlabel(r'$q$')
    ax1.set_xlim(0,5)
    ax1.set_ylabel(r'$-\langle \nabla \cdot \Pi_{r \theta} \rangle_{CG}$ in $%.2f < r/a < %.2f$' %(q['rho'][irhomin], q['rho'][irhomax]) )
    
    qrange = np.linspace(0,5,100)
    ax1.plot(qrange, 0.45e-7*(-1+qrange)**(1/3), c='g', label=r'$C (-1+q)^{1/3}$')
    
    ax1.legend()
    
    fig.savefig('Figures/qprof_divPi_08.pdf', format='pdf', bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\robin\AppData\Local\Temp/ipykernel_18288/1929424472.py:41: RuntimeWarning: invalid value encountered in power
  ax1.plot(qrange, 0.45e-7*(-1+qrange)**(1/3), c='g', label=r'$C (-1+q)^{1/3}$')


## <font color='red'>**Résultat**</font> - Profil radial des **flows ($E_r, V_P, V_T$)**

In [ ]:
plt.rcParams['axes.formatter.limits'] = (-5,6)
q_ar = [qprof_05,qprof_1,qprof_15]
#q_ar = [qprof_05_n0,qprof_15_n0]
param_ar = ['Phi00','Phi01','dPhi01dr','Er','VP','dErdr','RSpol','RSphi','dRSphidr','VT','nustar']
param_ar = ['Gamma_vD', 'Gamma_vE', 'Gamma_vEn0', 'Gamma_vEndiff0']

param_ar = ['Er','VT','VP']

title_ar = [r'$E_r[V_{T_0} B_0]$',r'$V_P$',r'$V_T$']

y_min_ar = [-2.5e-2, -6e-2, 4e-3]
y_max_ar = [-1e-2, 0, -3.5e-3]

for ip,p in enumerate(param_ar):
    fig = plt.figure(figsize=(8,8))
    #fig.suptitle(p)
    ax1 = fig.add_subplot(111)
    for iq,q in enumerate(q_ar):
        
        irhomin = (np.abs(q['rho'] - 0.1 )).argmin()
        irhomax = (np.abs(q['rho'] - 1.0 )).argmin()

        itmin = (np.abs(q['time'] - 100.0e3 )).argmin()
        itmax = (np.abs(q['time'] - 145.0e3 )).argmin()
        
        ax1.plot(q['rho'][irhomin:irhomax], np.mean( q[p][itmin:itmax,irhomin:irhomax],axis=0),ls=q['ls'],c=q['c'],lw=4,label=q['title'])
        #ax1.plot(q['rho'][irhomin:irhomax], np.mean( q[p][itmin:itmax,irhomin:irhomax],axis=0) /q['q'][irhomin:irhomax],ls='--',c=q['c'],label=q['title'])
        
        ax1.axhline(y=0,ls='--',c='k')
        
    ax1.set_ylabel(title_ar[ip])
    ax1.ticklabel_format(axis='y',scilimits=(-1,2),useMathText=True)
        
    ax1.legend()
    ax1.set_xlabel(r'$r/a$')
    
    ax1.axvline(x=1,ls='--',c='k')
    #ax1.text(x=0.7,y=(y_min_ar[ip]+y_max_ar[ip])/2.,s=r"$\nearrow I_P$",fontsize=18,c='g')
    
    #ax1.annotate("", xy=(0.8, y_min_ar[ip]), xytext=(0.8, y_max_ar[ip]),arrowprops=dict(width=5,facecolor='none',edgecolor='g'),horizontalalignment='center', verticalalignment='center')
    
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    fig.savefig('Figures/rprof_'+p+'_scanq.pdf', format='pdf', bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## <font color='red'>**Résultat**</font> - Profil radial de la friction néoclassique $\nu_\theta$ empirique tiré de *Gianakon PoP 2002, p12, Eq(C16)*

In [ ]:
q_ar = [qprof_05,qprof_1,qprof_15]

fig = plt.figure(figsize=(7,7))
#fig.suptitle(p)
ax1 = fig.add_subplot(111)

N=60
def u1d(X):
    #return uniform_filter1d(X,size=N)
    return uniform_filter1d(uniform_filter1d(X,size=N),size=N)
    #return uniform_filter1d(uniform_filter1d(uniform_filter1d(X,size=N),size=N),size=N)
    #return uniform_filter1d(uniform_filter1d(uniform_filter1d(uniform_filter1d(X,size=N),size=N),size=N),size=N)

def g1d(X):
    return gaussian_filter1d(X,sigma=N)

for iq,q in enumerate(q_ar):
    irhomin = (np.abs(q['rho'] - 0.1 )).argmin()
    irhomax = (np.abs(q['rho'] - 1.0 )).argmin()

    itmin = (np.abs(q['time'] - 100.0e3 )).argmin()
    itmax = (np.abs(q['time'] - 145.0e3 )).argmin()

    #Calcul de la friction NC poloidale (cf Gianakon 2002)
    nutheta = (0.452 * np.sqrt(2*q['eps']) * q['nu_i'])/( (1+1.03*q['nustar']**(1/2) +0.31 * q['nustar']) * (1 + 0.66 * q['nustar'] * q['eps']**(3/2))) * (q['q']/q['eps'])**2

    ax1.plot(q['rho'][irhomin:irhomax], np.mean(nutheta[itmin:itmax,irhomin:irhomax],axis=0),ls=q['ls'],c=q['c'],label=q['title'])

    ax1.axhline(y=0,ls='--',c='k')

ax1.set_ylabel(r'$\nu_\theta$')

ax1.ticklabel_format(axis='y',scilimits=(-1,2),useMathText=True)

ax1.legend()
ax1.set_xlabel(r'$r/a$')

fig.tight_layout(rect=[0, 0.03, 1, 0.95])

#fig.savefig('Figures/rprof_Pionq_scanq.pdf', format='pdf', bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## <font color='red'>**Résultat**</font> - Profil radial de **l'intensité turbulente $\mathcal{I}$**

In [11]:
plt.rcParams['axes.formatter.limits'] = (-5,6)
q_ar = [qprof_05,qprof_1,qprof_15]

param_ar = ['turb_int_sq_avg']

title_ar = [r'$\mathcal{I}$']

fig = plt.figure(figsize=(7,7))
#fig.suptitle(p)
ax1 = fig.add_subplot(111)

for iq,q in enumerate(q_ar):
    irhomin = (np.abs(q['rho'] - 0.1 )).argmin()
    irhomax = (np.abs(q['rho'] - 1.0 )).argmin()

    itmin = (np.abs(q['time'] - 80.0e3 )).argmin()
    itmax = (np.abs(q['time'] - 100.0e3 )).argmin()

    ax1.plot(q['rho'][irhomin:irhomax], np.sqrt(np.mean( (q['turb_int_sq_avg'])[itmin:itmax,irhomin:irhomax],axis=0)),ls=q['ls'],c=q['c'],label=q['title'])

    ax1.axhline(y=0,ls='--',c='k')

ax1.set_ylabel(r'$\mathcal{I}$')
#ax1.ticklabel_format(axis='y',scilimits=(-1,2),useMathText=True)

ax1.legend()
ax1.set_xlabel(r'$r/a$')

ax1.axvline(x=1,ls='--',c='k')

fig.tight_layout(rect=[0, 0.03, 1, 0.95])

#fig.savefig('Figures/rprof_turb_intens_scanq.pdf', format='pdf', bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## <font color='red'>**Résultat**</font> - Profil radial de la **fréquence de collision ionique $\nu_i$**

In [ ]:
plt.rcParams['axes.formatter.limits'] = (-5,6)
q_ar = [qprof_05,qprof_1,qprof_15]

param_ar = ['turb_int_sq_avg']

title_ar = [r'$\mathcal{I}$']

fig = plt.figure(figsize=(6,6))
#fig.suptitle(p)
ax1 = fig.add_subplot(111)

for iq,q in enumerate(q_ar):
    irhomin = (np.abs(q['rho'] - 0.1 )).argmin()
    irhomax = (np.abs(q['rho'] - 1.0 )).argmin()

    itmin = (np.abs(q['time'] - 100.0e3 )).argmin()
    itmax = (np.abs(q['time'] - 145.0e3 )).argmin()

    ax1.plot(q['rho'][irhomin:irhomax], np.mean( (q['nu_i'])[itmin:itmax,irhomin:irhomax],axis=0),ls=q['ls'],c=q['c'],label=q['title'],lw=4)

    ax1.axhline(y=0,ls='--',c='k')

ax1.set_ylabel(r'$\nu_i$ $[\omega_{c0}]$')
#ax1.ticklabel_format(axis='y',scilimits=(-1,2),useMathText=True)

ax1.legend()
ax1.set_xlabel(r'$r/a$')

ax1.axvline(x=1,ls='--',c='k')

fig.tight_layout(rect=[0, 0.03, 1, 0.95])

fig.savefig('Figures/rprof_nu_i.pdf', format='pdf', bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## <font color='coral'>**Travail en cours**</font> - Transfert d'énergie de la turbulence vers les écoulements via la **puissance $\mathcal{P} = \langle - \nabla \cdot \Pi_{r\theta} \rangle V_E$**

In [23]:
#import panel as pn
#pn.extension()

plt.close('all')
import matplotlib.colors as colors

q_ar = [qprof_05,qprof_15]

for iq,q in enumerate(q_ar):
        q['Ptransfert'] = -q['dRSpoldr'] * (-q['Er'])

symetrical_cmap = 0

def update_plot(N,colormap_range,float_range_time,float_range_rho):
    tmin_avg,tmax_avg = float_range_time
    rhomin_avg,rhomax_avg = float_range_rho
    
    plt.close('all')
    
    fig = plt.figure(figsize=(xfigsize,10))
    fig.suptitle(r'Study of $P = \langle - \nabla \cdot \Pi_{r \theta} \rangle V_{E \theta}$')

    Pmap = uniform_filter1d(uniform_filter1d(qprof_05['Ptransfert'],size=N,axis=1),size=N,axis=1)
    if symetrical_cmap:
        vmin = colormap_range*min( abs( np.min(Pmap) ), abs( np.max(Pmap) ) ) 
        vmax = -vmin
    else:
        vmin = colormap_range*np.min(Pmap) 
        vmax = colormap_range*np.max(Pmap) 

    gs = gridspec.GridSpec(3, 2*len(q_ar))
    
    for iq,q in enumerate(q_ar):

        irhomin = (np.abs(q['rho'] - 0.0 )).argmin()
        irhomax = (np.abs(q['rho'] - 1.1 )).argmin()
        
        tmin = 100e3 ; tmax = 150e3
        itmin = (np.abs(q['time'] - tmin  )).argmin()
        itmax = (np.abs(q['time'] - tmax )).argmin()

        p_ma = uniform_filter1d(uniform_filter1d(q['Ptransfert'],size=N,axis=1),size=N,axis=1)
        
        ## Figure 1
        ax_map = fig.add_subplot(gs[0:2, 2*iq:2*iq+2]) 

        ax_map.set_title(q['title'])
        p=ax_map.pcolormesh(q['rho'][irhomin:irhomax],q['time'][itmin:itmax],p_ma[itmin:itmax,irhomin:irhomax],vmin=vmin,vmax=vmax)
        fig.colorbar(p,ax=ax_map)
        fig.tight_layout()
        ax_map.set_xlabel(r'$r/a$')
        ax_map.set_ylabel(r'Time $[\omega_{c0}^{-1}]$')

        ax_map.axvline(x=1,c='k',ls='--')
        for rline in [0.8,0.6,0.4,0.2]:ax_map.axvline(x=rline,c='k',ls='--')
        ax_map.set_xticks([1.0,0.8,0.6,0.4,0.2])
        
        ax_map.axvline(x=rhomin_avg,c='g',ls='-')
        ax_map.axvline(x=rhomax_avg,c='g',ls='-')
        
        ax_map.axhline(y=tmin_avg,c='r',ls='-')
        ax_map.axhline(y=tmax_avg,c='r',ls='-')
        
        ax_map.plot(q['rho'][10:10+N],np.ones(N)*((tmax-tmin)/2+tmin),c='xkcd:battleship grey',lw=5) #Visualize the sliding average window

        
        ##Figure 2
        itmin_avg = (np.abs(q['time'] - tmin_avg )).argmin()
        itmax_avg = (np.abs(q['time'] - tmax_avg )).argmin()

        ax_rprof = fig.add_subplot(gs[2:, 0:len(q_ar)]) 
        ax_rprof.set_title(r'Radial profile (avg %.0f < t$[\omega_{c0}^{-1}]$ < %.0f)' % (tmin_avg , tmax_avg) )
        ax_rprof.plot(q['rho'][irhomin:irhomax], np.mean( p_ma[itmin_avg:itmax_avg,irhomin:irhomax],axis=0),ls=q['ls'],c=q['c'],label=q['title'])
        ax_rprof.set_xlabel(r'$r/a$')
        ax_rprof.axhline(y=0,c='k',ls='--')
        ax_rprof.legend()
        
        irhomin_avg = (np.abs(q['rho'] - rhomin_avg )).argmin()
        irhomax_avg = (np.abs(q['rho'] - rhomax_avg )).argmin()

        ax_tevol = fig.add_subplot(gs[2:, len(q_ar):]) 
        ax_tevol.set_title(r'Temporal evolution (avg %.2f < r/a < %.2f)' % (rhomin_avg , rhomax_avg) )
        ax_tevol.plot(q['time'][itmin:itmax], np.mean(p_ma[itmin:itmax,irhomin_avg:irhomax_avg],axis=1),ls=q['ls'],c=q['c'],label=q['title'])
        ax_tevol.set_xlabel(r'Time $[\omega_{c0}^{-1}]$')
        ax_tevol.axhline(y=0,c='k',ls='--')
        ax_tevol.legend()

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

float_range_time_slider = widgets.FloatRangeSlider(value=[100e3, 145e3], min=0, max=150e3, step=1000, description='Time range', layout=Layout(width='50%'), style = {'description_width': 'initial'})
    
float_range_rho_slider = widgets.FloatRangeSlider(value=[0.75, 0.85], min=0, max=1.2, step=0.05, description='Normalized radius range', layout=Layout(width='50%'), style = {'description_width': 'initial'})
    
interact(update_plot, 
         N=widgets.IntSlider(min=1, max=100, step=1,value=1),
         float_range_time=float_range_time_slider,
         float_range_rho=float_range_rho_slider,
         colormap_range=widgets.FloatSlider(min=0.1, max=2, step=0.05,value=0.8)
        )

interactive(children=(IntSlider(value=1, description='N', min=1), FloatSlider(value=0.8, description='colormap…

<function __main__.update_plot(N, colormap_range, float_range_time, float_range_rho)>

## <font color='coral'>**Travail en cours**</font> - Profil radial de la coarse-grained **puissance $\mathcal{P} = \langle - \nabla \cdot \Pi_{r\theta} \rangle V_E$**

In [ ]:
q_ar = [qprof_05,qprof_15]
param_ar = ['Phi00','Phi01','dPhi01dr','Er','VP','dErdr','RSpol','RSphi','dRSphidr','VT']

param_ar = ['dRSpoldr']

for iq,q in enumerate(q_ar):
    q['Ptransfert'] = -q['dRSpoldr'] * (-q['Er'])
    q['Ptransfert_approx'] = -q['RSpol'] * (-q['dErdr'])
    q['Ptransfert_res'] = np.gradient(-q['RSpol'] * (-q['Er']), q['rg'], axis=1) 

param_ar = ['Ptransfert']
    
for ip,p in enumerate(param_ar):
    
    def update_plot(N):
        plt.close('all')
    
        fig = plt.figure(figsize=(xfigsize,6))
        fig.suptitle(p)
        ax1 = fig.add_subplot(111)
        size_window = N * 250 * 1.3 / len(qprof_05['rg']) 
        ax1.set_title(r'Approximative window size for avg : %.1f $\rho_i$' % size_window )
        for iq,q in enumerate(q_ar):

            irhomin = (np.abs(q['rho'] - 0. )).argmin()
            irhomax = (np.abs(q['rho'] - 1. )).argmin()

            itmin = (np.abs(q['time'] - 60e3 )).argmin()
            itmax = (np.abs(q['time'] - 80e3 )).argmin()

            ## Plot raw
            ax1.plot(q['rho'][irhomin:irhomax], np.mean(q[p][itmin:itmax,irhomin:irhomax],axis=0),c=q['c'], ls='--')
            
            ## Plot coarse-grained
            p_ma = uniform_filter1d(uniform_filter1d(np.mean(q[p][itmin:itmax,irhomin:irhomax],axis=0),size=N),size=N)

            ax1.plot(q['rho'][irhomin:irhomax], p_ma, label=q['title'],lw=3,c=q['c'], ls='-')

            ax1.axhline(y=0,c='k',ls='--')

        ax1.plot(q['rho'][10:10+N],np.ones(N)*np.amax(p_ma)/2,c='xkcd:battleship grey',lw=5)

        ax1.legend()
        ax1.set_xlabel(r'$r/a$')
    
    interact(update_plot, N=(1, 100, 1), value=60);

interactive(children=(IntSlider(value=50, description='N', min=1), Output()), _dom_classes=('widget-interact',…

## <font color='red'>**Résultat**</font> - Validation de la simplification sur la **puissance $\mathcal{P} = \langle - \nabla \cdot \Pi_{r\theta} \rangle V_E$**

In [22]:
q_ar = [qprof_05,qprof_15]

for iq,q in enumerate(q_ar):
    #q['Ptransfert'] = -q['dRSpoldr'] * (-q['Er'])
    q['Ptransfert_approx'] = q['RSpol'] * (-q['dErdr'])
    q['Ptransfert_res'] = np.gradient(-q['RSpol'] * (-q['Er']), q['rg'], axis=1) 

param_ar = ['Ptransfert', 'Ptransfert_approx']
label_ar = [r'$\nabla \cdot \Pi_{r\theta} \times V_{E\theta}$', r'$\Pi_{r\theta} \times \partial_r V_{E\theta}$']
    
colors_ar = ['xkcd:blue','xkcd:red','xkcd:purple','xkcd:green']
colors_ar2 = ['xkcd:blue green','xkcd:dark orange','xkcd:bright pink','xkcd:apple green']
    
def update_plot(N,dpi_adjust):
    plt.close('all')

    fig = plt.figure(figsize=(6*len(q_ar),6), dpi=dpi_adjust)
    size_window = N * 250 * 1.3 / len(qprof_05['rg']) 
    fig.suptitle(r'Approximative window size for avg : %.1f $\rho_i$' % size_window )
    for iq,q in enumerate(q_ar):

        irhomin = (np.abs(q['rho'] - 0. )).argmin()
        irhomax = (np.abs(q['rho'] - 1. )).argmin()

        itmin = (np.abs(q['time'] - 100e3 )).argmin()
        itmax = (np.abs(q['time'] - 145e3 )).argmin()

        ax1 = fig.add_subplot(1,len(q_ar),iq+1)
        ax1.set_title(q['title'])
        for ip,p in enumerate(param_ar):
            ## Plot raw
            ax1.plot(q['rho'][irhomin:irhomax], np.mean(q[p][itmin:itmax,irhomin:irhomax],axis=0), c=colors_ar[ip], lw=1.5, ls='--')

            ## Plot coarse-grained
            ax1.plot(q['rho'][irhomin:irhomax], uniform_filter1d(uniform_filter1d(np.mean(q[p][itmin:itmax,irhomin:irhomax],axis=0),size=N),size=N), label=label_ar[ip],c=colors_ar[ip],lw=3, ls='-')

        ax1.axhline(y=0,c='k',ls='--')
        ax1.legend()
        ax1.set_xlabel(r'$r/a$')
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    ax1.plot(q['rho'][10:10+N],np.ones(N)*ax1.get_ylim()[1]/2,c='xkcd:battleship grey',lw=5)

interact(update_plot, 
         N=widgets.IntSlider(min=1, max=100, step=1,value=60),
         dpi_adjust=widgets.FloatSlider(min=30, max=200, step=1,value=100));

interactive(children=(IntSlider(value=60, description='N', min=1), FloatSlider(value=100.0, description='dpi_a…

## <font color='coral'>**Travail en cours**</font> - Profil des termes dans l'équation $\frac{1}{2} \partial_t V_E^2 = - \langle \Pi_{r\theta} \rangle V_E - \nu_\theta (V_E - V_{neo}) V_E$
/!\ Ici on estime $- \nu_\theta (V_E - V_{neo}) V_E = \frac{1}{2} \partial_t V_E^2 + \langle \Pi_{r\theta} \rangle V_E$ donc on ne vérifie pas une conservation, au contraire on estime qu'elle est toujours vérifiée ce qui permet d'estimer le terme néoclassique

In [11]:
def update_plot(dpi_adjust,float_range_time,
               q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool,
               q05br_bool,q075br_bool,q15br_bool,
               prefactor_bool):

   tmin_avg,tmax_avg = float_range_time

   q_ar = [qprof for qprof, qprof_bool in zip(all_sim, [q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool,q05br_bool,q075br_bool,q15br_bool]) if qprof_bool]

   fig = plt.figure(figsize=(6*len(q_ar),12), dpi=dpi_adjust)

   for iq,q in enumerate(q_ar):

      irhomin = (np.abs(q['rho'] - 0.0 )).argmin()
      irhomax = (np.abs(q['rho'] - 1.1 )).argmin()

      itmin = (np.abs(q['time'] - tmin_avg )).argmin()
      itmax = (np.abs(q['time'] - tmax_avg )).argmin()

      if prefactor_bool:
         term1 = q['tevol_VEsq']
         term2 = q['neo_energy_evol_w_prefactor']
         term3 = q['Ptransfert_w_prefactor']
      else :
         term1 = q['tevol_VEsq']
         term2 = q['neo_energy_evol']
         term3 = q['Ptransfert']

      ax = fig.add_subplot(len(q_ar),1,iq+1)
      ax.set_title(q['title'])
      ax.plot(q['rho'][irhomin:irhomax], np.mean(term1[itmin:itmax,irhomin:irhomax],axis=0), label=r'$ \frac{1}{2}\partial_t V_E^2$',c='xkcd:blue',lw=3, ls='-')
      ax.plot(q['rho'][irhomin:irhomax], np.mean(term2[itmin:itmax,irhomin:irhomax],axis=0), label=r'$- \nu_\theta (V_E - V_{neo}) V_E$',c='xkcd:red',lw=3, ls='-')
      ax.plot(q['rho'][irhomin:irhomax], np.mean(term3[itmin:itmax,irhomin:irhomax],axis=0), label=r'$-\langle \Pi_{r\theta} \times V_{E\theta} \rangle$',c='xkcd:purple',lw=3, ls='-')
      ax.axhline(y=0,c='k',ls='--')
      ax.legend(loc=('center left'))
      ax.set_xlabel(r'$r/a$')
      ax.axvline(x=1,c='k',ls='--')
   fig.tight_layout(rect=[0, 0.03, 1, 0.95])
   
float_range_time_slider = widgets.FloatRangeSlider(value=[100e3, 145e3], min=0, max=150e3, step=1000, description='Time range', layout=Layout(width='50%'), style = {'description_width': 'initial'})


interact(update_plot, 
         float_range_time=float_range_time_slider,
         dpi_adjust=widgets.FloatSlider(min=30, max=100, step=1,value=dpi_custom),
         q05_bool    = widgets.Checkbox(value=True,description='q05'),
         q1_bool     = widgets.Checkbox(value=False,description='q1'),
         q15_bool    = widgets.Checkbox(value=True,description='q15'),
         q05n0_bool  = widgets.Checkbox(value=False,description='q05n0'),
         q15n0_bool  = widgets.Checkbox(value=False,description='q15n0'),
         q05new_bool = widgets.Checkbox(value=False,description='q05new'),
         q05br_bool = widgets.Checkbox(value=False,description='q05br'),
         q075br_bool = widgets.Checkbox(value=False,description='q075br'),
         q15br_bool = widgets.Checkbox(value=False,description='q15br'),
         prefactor_bool = widgets.Checkbox(value=True,description=r'With prefactor $1+2q^2$')
        )

interactive(children=(FloatSlider(value=100.0, description='dpi_adjust', min=30.0, step=1.0), FloatRangeSlider…

<function __main__.update_plot(dpi_adjust, float_range_time, q05_bool, q1_bool, q15_bool, q05n0_bool, q15n0_bool, q05new_bool, q05br_bool, q075br_bool, q15br_bool, prefactor_bool)>

## <font color='coral'>**Travail en cours**</font> - Estimation de $\nu_\theta$ sur les simus néoclassiques

In [28]:
def update_plot(dpi_adjust,float_range_time,float_range_rho,
               q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool):

   tmin_avg,tmax_avg = float_range_time
   rhomin_avg,rhomax_avg = float_range_rho

   q_ar = []
   if q05_bool    : q_ar = np.append(q_ar,qprof_05)
   if q1_bool     : q_ar = np.append(q_ar,qprof_1)
   if q15_bool    : q_ar = np.append(q_ar,qprof_15)
   if q05n0_bool  : q_ar = np.append(q_ar,qprof_05_n0)
   if q15n0_bool  : q_ar = np.append(q_ar,qprof_15_n0)
   if q05new_bool : q_ar = np.append(q_ar,qprof_05new)

   fig = plt.figure(figsize=(6*len(q_ar),12), dpi=dpi_adjust)

   ax  = fig.add_subplot(111)

   for iq,q in enumerate(q_ar):

      irhomin = (np.abs(q['rho'] - 0.0 )).argmin()
      irhomax = (np.abs(q['rho'] - 1.1 )).argmin()

      itmin = (np.abs(q['time'] - tmin_avg )).argmin()
      itmax = (np.abs(q['time'] - tmax_avg )).argmin()

      #ax = fig.add_subplot(len(q_ar),1,iq+1)
      ax.set_title(q['title'])
      ax.plot(np.mean(q['Er'][itmin:itmax,irhomin:irhomax],axis=1), 
              np.mean(q['dErdr'][itmin:itmax,irhomin:irhomax],axis=1), 
              label=q['title'],c=q['c'],lw=3, ls='-')
      #ax.axhline(y=0,c='k',ls='--')
      ax.legend()
      ax.set_xlabel(r'$E_r$')
   fig.tight_layout(rect=[0, 0.03, 1, 0.95])
   
float_range_time_slider = widgets.FloatRangeSlider(value=[10e3, 70e3], min=0, max=150e3, step=1000, description='Time range', layout=Layout(width='50%'), style = {'description_width': 'initial'})

float_range_rho_slider = widgets.FloatRangeSlider(value=[0.84, 0.86], min=0, max=1.2, step=0.01, description='Normalized radius range', layout=Layout(width='50%'), style = {'description_width': 'initial'})
    

interact(update_plot, 
         float_range_time=float_range_time_slider,
         float_range_rho=float_range_rho_slider,
         dpi_adjust=widgets.FloatSlider(min=30, max=100, step=1,value=dpi_custom),
         q05_bool    = widgets.Checkbox(value=False,description='q05'),
         q1_bool     = widgets.Checkbox(value=False,description='q1'),
         q15_bool    = widgets.Checkbox(value=False,description='q15'),
         q05n0_bool  = widgets.Checkbox(value=True,description='q05n0'),
         q15n0_bool  = widgets.Checkbox(value=True,description='q15n0'),
         q05new_bool = widgets.Checkbox(value=False,description='q05new')
        )

interactive(children=(FloatSlider(value=65.0, description='dpi_adjust', min=30.0, step=1.0), FloatRangeSlider(…

<function __main__.update_plot(dpi_adjust, float_range_time, float_range_rho, q05_bool, q1_bool, q15_bool, q05n0_bool, q15n0_bool, q05new_bool)>

## <font color='royalblue'>**Temporaire**</font> - Fonction porte "smooth"

In [ ]:
def smoothed_rectangular(x, left, right, k=10):
    """
    Returns a smoothed rectangular function with smoothed discontinuities.
    
    Parameters:
        x (array-like): The independent variable.
        left (float): The left edge of the rectangular function.
        right (float): The right edge of the rectangular function.
        k (float): Controls the steepness of the sigmoid function. Higher k leads to sharper transition.
        
    Returns:
        y (array-like): The smoothed rectangular function evaluated at x.
    """
    y = 1 / (1 + np.exp(-k * (x - left))) - 1 / (1 + np.exp(-k * (x - right)))
    return y

# Example usage
x = np.linspace(-10, 10, 1000)
y = smoothed_rectangular(x, -9, 9, k=10)
plt.figure()
plt.plot(x, y)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## <font color='xkcd:orange'>**(OLD)**</font> <font color='xkcd:sage'>**Exploration**</font> - Etude dans l'espace des fréquences

In [72]:
plt.close('all')
import matplotlib.colors as colors


symetrical_cmap = 0

param= 'Ptransfert'

def update_plot(N,colormap_range,dpi_adjust,param,float_range_time,float_range_rho,
               q05_bool,q1_bool,q15_bool,q05n0_bool,q15n0_bool,q05new_bool):
    tmin_avg,tmax_avg = float_range_time
    rhomin_avg,rhomax_avg = float_range_rho
    
    q_ar = []
    if q05_bool    : q_ar = np.append(q_ar,qprof_05)
    if q1_bool     : q_ar = np.append(q_ar,qprof_1)
    if q15_bool    : q_ar = np.append(q_ar,qprof_15)
    if q05n0_bool  : q_ar = np.append(q_ar,qprof_05_n0)
    if q15n0_bool  : q_ar = np.append(q_ar,qprof_15_n0)
    if q05new_bool : q_ar = np.append(q_ar,qprof_05new)
    
    plt.close('all')
    
    fig = plt.figure(figsize=(16,10), dpi=dpi_adjust)
    fig.suptitle(r'Study of %s' %param)

    Pmap = uniform_filter1d(uniform_filter1d(qprof_05[param],size=N,axis=1),size=N,axis=1)
    if symetrical_cmap:
        vmin = colormap_range*min( abs( np.min(Pmap) ), abs( np.max(Pmap) ) ) 
        vmax = -vmin
    else:
        vmin = colormap_range*np.min(Pmap) 
        vmax = colormap_range*np.max(Pmap) 

    gs = gridspec.GridSpec(5, 2*len(q_ar))
    
    for iq,q in enumerate(q_ar):

        irhomin = (np.abs(q['rho'] - 0.0 )).argmin()
        irhomax = (np.abs(q['rho'] - 1.1 )).argmin()

        itmin = (np.abs(q['time'] - 0e3  )).argmin()
        itmax = (np.abs(q['time'] - 145e3 )).argmin()

        p_ma = uniform_filter1d(uniform_filter1d(q[param],size=N,axis=1),size=N,axis=1)
        
        ## Figure 1
        ax_map = fig.add_subplot(gs[0:3, 2*iq:2*iq+2]) 

        ax_map.set_title(q['title'])
        p=ax_map.pcolormesh(q['rho'][irhomin:irhomax],q['time'][itmin:itmax],p_ma[itmin:itmax,irhomin:irhomax],vmin=vmin,vmax=vmax)
        fig.colorbar(p,ax=ax_map)
        fig.tight_layout()
        ax_map.set_xlabel(r'$r/a$')
        ax_map.set_ylabel(r'Time $[\omega_{c0}^{-1}]$')

        ax_map.axvline(x=1,c='k',ls='--')
        for rline in [0.8,0.6,0.4,0.2]:ax_map.axvline(x=rline,c='k',ls='--')
        ax_map.set_xticks([1.0,0.8,0.6,0.4,0.2])
        
        ax_map.axvline(x=rhomin_avg,c='g',ls='-')
        ax_map.axvline(x=rhomax_avg,c='g',ls='-')
        
        ax_map.axhline(y=tmin_avg,c='r',ls='-')
        ax_map.axhline(y=tmax_avg,c='r',ls='-')
        
        ax_map.plot(q['rho'][10:10+N],np.ones(N)*ax_map.get_ylim()[1]/4,c='xkcd:battleship grey',lw=5) #Visualisation de la fenêtre de la moyenne glissante

        
        ##Figure 2
        itmin_avg = (np.abs(q['time'] - tmin_avg )).argmin()
        itmax_avg = (np.abs(q['time'] - tmax_avg )).argmin()

        irhomin_avg = (np.abs(q['rho'] - rhomin_avg )).argmin()
        irhomax_avg = (np.abs(q['rho'] - rhomax_avg )).argmin()
        
        #Perform fft
        dt = q['time'][-1]-q['time'][-2]
        w = np.fft.fftfreq(q['time'][itmin_avg:itmax_avg].shape[-1], d=dt) #Calcul des fréquences.
        fspectra = np.fft.fft(p_ma[itmin_avg:itmax_avg,:], axis=0)[np.argsort(w)]*dt/(tmax_avg-tmin_avg) #Calcul du spectre. La fonction argsort permet d'organiser les éléments du spectre dans l'ordre tel que les fréquences soient trier dans l'ordre croissant (pas indispensable mais utile pour la visualisation) 

        
        ax_fspectra = fig.add_subplot(gs[3:, 0:len(q_ar)]) 
        ax_fspectra.set_title(r'Fourier amplitude (avg %.2f < r/a < %.2f)' % (rhomin_avg , rhomax_avg) )
        ax_fspectra.semilogy(np.sort(w), np.mean(np.abs(fspectra[:,irhomin_avg:irhomax_avg]),axis=1),ls=q['ls'],c=q['c'],label=q['title'])
        ax_fspectra.set_xlabel(r'$\omega [\omega_{c0}]$')
        ax_fspectra.legend(fontsize=14)
        
        #Fréquence GAM théorique
        #omega_theo = np.mean( np.mean( (2*np.sqrt(q['T'] * (5/3) *(1 + (1/(2*q['q']**2) ) )) /800)[itmin_avg:itmax_avg] ,axis=0)[irhomin_avg:irhomax_avg] )
        #ax_fspectra.axvline(x=omega_theo,c='purple')
        
        ax_fphase = fig.add_subplot(gs[3:, len(q_ar):]) 
        ax_fphase.set_title(r'Fourier phase (avg %.2f < r/a < %.2f)' % (rhomin_avg , rhomax_avg) )
        ax_fphase.plot(np.sort(w), np.mean(np.angle(fspectra[:,irhomin_avg:irhomax_avg]),axis=1),ls=q['ls'],c=q['c'],label=q['title'])
        ax_fphase.set_xlabel(r'$\omega [\omega_{c0}]$')
        ax_fphase.axhline(y=0,ls='--',c='k')
        ax_fphase.legend(fontsize=16)

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

float_range_time_slider = widgets.FloatRangeSlider(value=[63e3, 101e3], min=0, max=150e3, step=1000, description='Time range', layout=Layout(width='50%'), style = {'description_width': 'initial'})
    
float_range_rho_slider = widgets.FloatRangeSlider(value=[0.75, 0.85], min=0, max=1.2, step=0.01, description='Normalized radius range', layout=Layout(width='50%'), style = {'description_width': 'initial'})
    
interact(update_plot, 
         N=widgets.IntSlider(min=1, max=100, step=1,value=1),
         float_range_time=float_range_time_slider,
         float_range_rho=float_range_rho_slider,
         colormap_range=widgets.FloatSlider(min=0.1, max=2, step=0.05,value=0.8),
         dpi_adjust=widgets.FloatSlider(min=30, max=100, step=1,value=100),
         param = widgets.Text(value='Ptransfert',placeholder='Type something',description='Select parameter:',disabled=False),
         q05_bool    = widgets.Checkbox(value=True,description='q05'),
         q1_bool     = widgets.Checkbox(value=False,description='q1'),
         q15_bool    = widgets.Checkbox(value=True,description='q15'),
         q05n0_bool  = widgets.Checkbox(value=False,description='q05n0'),
         q15n0_bool  = widgets.Checkbox(value=False,description='q15n0'),
         q05new_bool = widgets.Checkbox(value=False,description='q05new')
        )

interactive(children=(IntSlider(value=1, description='N', min=1), FloatSlider(value=0.8, description='colormap…

<function __main__.update_plot(N, colormap_range, dpi_adjust, param, float_range_time, float_range_rho, q05_bool, q1_bool, q15_bool, q05n0_bool, q15n0_bool, q05new_bool)>